In [1]:
import pandas as pd

In [161]:
#import annotazioni e gold standard

#annotazioni= pd.read_csv('https://raw.githubusercontent.com/GConte47/tirocinio/refs/heads/master/df%20omap/files/annotazioni_full.csv', delimiter=',')
#GS= pd.read_csv('https://raw.githubusercontent.com/GConte47/tirocinio/refs/heads/master/df%20omap/files/gold_standard.csv', delimiter=',')

#import annotazioni e gold standard
annotazioni= pd.read_csv('../files/annotazioni_full.csv', delimiter=',')
GS= pd.read_csv('../files/gold_standard.csv', delimiter=',')


print(annotazioni.shape[0], GS.shape[0])

38102 323


# descrizione problema

In [162]:
annotazioni.columns

Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui', 'annotator'],
      dtype='object')

In [163]:
GS.columns

Index(['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui'],
      dtype='object')

In [164]:
#allineazione nomi
annotazioni = annotazioni.rename(columns={'ontologies':'Ontologies','SOURCE':'Source','entity_id':'EntityID','pretty_name':'PrettyName','cui':'Cui','annotator':'Annotator',})
annotazioni['Ontologies'] = annotazioni['Ontologies'].replace('SNOMEDCT', 'SNOMED-CT')

GS = GS.rename(columns={'entity_id':'EntityID','pretty_name':'PrettyName','cui':'Cui','annotator':'Annotator',})

In [165]:

set_gold = set(GS[['Ontologies', 'Cui']].itertuples(index=False, name=None))
set_full = set(annotazioni[['Ontologies', 'Cui']].itertuples(index=False, name=None))

# Calcoliamo i valori richiesti
Nboth = len(set_gold & set_full)  # Presenti in entrambi
Nleft = len(set_gold - set_full)  # Solo in df_gold
Nright = len(set_full - set_gold)  # Solo in df_full

print(f"Nboth: {Nboth}, Nleft: {Nleft}, Nright: {Nright}")

Nboth: 58, Nleft: 34, Nright: 1345


In [166]:
pd.DataFrame(list(set_gold-set_full), columns=['Ontologies', 'Cui']).head(5)

,Ontologies,Cui
0,UMLS,C0870134
1,UMLS,C1300638
2,SNOMED-CT,396278008
3,UMLS,C0079170
4,UMLS,C1627936


In [158]:
#dettagliare questa analisi rispetto alle diverse ontologie

set_GS = set(GS[['Ontologies', 'Cui']].itertuples(index=False, name=None))
set_annotazioni = set(annotazioni[['Ontologies', 'Cui']].itertuples(index=False, name=None))

# Unione dei due DataFrame per analisi per Ontology
df_combined = pd.concat([GS.assign(source='GS'), annotazioni.assign(source='annotazioni')])

# Creiamo una tabella che conta Nleft, Nright e Nboth per ogni Ontology
ontology_counts = df_combined.groupby(['Ontologies', 'source']).size().unstack(fill_value=0)
ontology_counts.columns = ['Nleft', 'Nright']
ontology_counts['Nboth'] = ontology_counts.min(axis=1)  # Il minimo tra Nleft e Nright dà Nboth

# Output finale
print(ontology_counts)

            Nleft  Nright  Nboth
Ontologies                      
LOINC           0    7903      0
SNOMED-CT     127   20398    127
UMLS          196    9801    196


In [9]:
# ?
# quindi non abbiamo nessun elemento di LOINC in comune ?

Non abbiamo elementi in comune con LOINC perchè lato MedCAT non abbiamo codici LOINC, stessa cosa nel gold standard perchè siccome non era previsto LOINC, abbiamo deciso di non inserirlo nel gold standard.

In [10]:
## faccio lo stesso controllo su ['Source', 'AttributoNomeCompleto']


Aset_gold = set(GS[['Source', 'AttributoNomeCompleto']].itertuples(index=False, name=None))
Aset_full = set(annotazioni[['Source', 'AttributoNomeCompleto']].itertuples(index=False, name=None))

# Calcoliamo i valori richiesti
ANboth = len(Aset_gold & Aset_full)  # Presenti in entrambi
ANleft = len(Aset_gold - Aset_full)  # Solo in df_gold
ANright = len(Aset_full - Aset_gold)  # Solo in df_full

print(f"Nboth: {ANboth}, Nleft: {ANleft}, Nright: {ANright}")

Nboth: 41, Nleft: 33, Nright: 671


In [11]:
# ?
# ma se non ho sbagliato i calcoli abbiamo solo 41 attributi in comune su cui posso fare il confronto
# ce ne sono 33 solo nel gold standard? forse c'è ancora quel problema sui nomi con i -

Nono, nessun errore, sono 33 i valori che non vengono usati perchè non sono presenti nel dataset annotazioni

In [159]:
set_gold = set(GS[['Ontologies', 'Cui']].itertuples(index=False, name=None))
set_full = set(annotazioni[['Ontologies', 'Cui']].itertuples(index=False, name=None))

Aset_gold = set(GS[['Source', 'AttributoNomeCompleto']].itertuples(index=False, name=None))
Aset_full = set(annotazioni[['Source', 'AttributoNomeCompleto']].itertuples(index=False, name=None))


GS_both = GS[
    GS.set_index(['Ontologies', 'Cui']).index.isin(set_gold & set_full) &
    GS.set_index(['Source', 'AttributoNomeCompleto']).index.isin(Aset_gold & Aset_full)
]
annotazioni_both = annotazioni[
    annotazioni.set_index(['Ontologies', 'Cui']).index.isin(set_gold & set_full) &
    annotazioni.set_index(['Source', 'AttributoNomeCompleto']).index.isin(Aset_gold & Aset_full)
]
print(GS_both.shape[0], annotazioni_both.shape[0])

134 943


In [160]:
annotazioni_both

,TipoDiAnnotazione,Ontologies,Source,AttributoNomeCompleto,EntityID,PrettyName,Cui,Annotator
1281,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,OMOP,person-person_id,0,Person,125676002,MedCAT
1286,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,OMOP,person-person_id,5,Source,260753009,MedCAT
1289,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,OMOP,person-person_id,10,Source,260753009,MedCAT
1291,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,OMOP,person-person_id,13,Person,125676002,MedCAT
1292,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,OMOP,person-gender_concept_id,0,Person,125676002,MedCAT
...,...,...,...,...,...,...,...,...
37879,BioportalAttributo,SNOMED-CT,OMOP,location-address_1,5,Address,397635003,Bioportal
37881,BioportalAttributo,SNOMED-CT,OMOP,location-address_2,5,Address,397635003,Bioportal
37882,BioportalAttributo,SNOMED-CT,OMOP,location-state,3,State,398070004,Bioportal
37884,BioportalAttributo,SNOMED-CT,OMOP,location-location_source_value,4,Location,246267002,Bioportal


possiamo sfruttare 134 record GS per controllare 943 record di annotazioni

## 1) da inserire nella tesi

DA INSERIRE NELLA TESI IL DISCORSO E LA TABELLA


Nel gold standard abbiamo annotato **74 attributi distinti**.  
Tra questi:

- Tutti i **74 attributi** sono stati annotati utilizzando concetti della **ontologia UMLS**, coprendo quindi la totalità.
- Solo **68 attributi** sono stati invece annotati con concetti della **ontologia SNOMED-CT**.

La tabella seguente riporta anche il **numero totale di annotazioni**:

- Per **UMLS** sono presenti **196 annotazioni**, con una media di circa **2.65 annotazioni per attributo** (196 diviso 74).
- Per **SNOMED-CT** le annotazioni totali sono **127**, con una media di circa **1.87 annotazioni per attributo** (127 diviso 68).

In [13]:
# Totale distinti AttributoNomeCompleto
totale_distinti = GS['AttributoNomeCompleto'].nunique()
print(totale_distinti)


# Group by con due aggregazioni: distinti e non distinti
risultato = (
    GS
    .groupby(['TipoDiAnnotazione', 'Ontologies'])
    .agg(
        NumAttributiDistinti=('AttributoNomeCompleto', pd.Series.nunique),
        NumAnnotazioniTotali=('AttributoNomeCompleto', 'count')
    )
    .reset_index()
)

# Aggiunta percentuale rispetto al totale
risultato['PercSulTotale'] = (risultato['NumAttributiDistinti'] / totale_distinti * 100).round(2)

risultato

74


,TipoDiAnnotazione,Ontologies,NumAttributiDistinti,NumAnnotazioniTotali,PercSulTotale
0,MedicalGS,SNOMED-CT,68,127,91.89
1,MedicalGS,UMLS,74,196,100.00


## 2) da inserire nella tesi

DA INSERIRE NELLA TESI IL DISCORSO E LA TABELLA

Le annotazioni oggetto di valutazioni sono quelle prodotte dagli annotatori per le quali abbiamo anche il relativo Gold Standard; esse sono contenute in un  dataframe che chiameremo *annotazioni_both*
Abbiamo in totale **otto annotazioni**, prodotte da due   *annotatori*:

- **Quattro annotazioni** sono state effettuate tramite **MEDCAT**:
  - Le prime **tre** utilizzano l'ontologia **SNOMED-CT**
  - La **quarta** utilizza l'ontologia **UMLS**

- Le altre **quattro annotazioni** sono state effettuate tramite **BioPortal**, tutte utilizzando **SNOMED-CT**

OVVIAMENTE FARE RIFERIMENTO AL CAPITOLO/SEZIONE DELLA TESI DOVE QUESTE TIPOLOGIE DI ANNOTAZIONI SONO STATE DESCRITTE

In [14]:
annotazioni_both['TipoDiAnnotazione'].unique().tolist()

['Medcat_SNOMED_Df_CS_08_ANC',
 'Medcat_SNOMED_Df_CS_1_ANC',
 'Medcat_SNOMED_Df_Full_ANC',
 'Medcat_UMLS_Df_Full_ANC',
 'BioportalAttributoNomeCompleto',
 'BioportalDescrizioneCompleta',
 'BioportalDescrizioneAttributo',
 'BioportalAttributo']

RIPORTARE NELLA TESI DISCORSO E TABELLA

La tabella seguente riassume i risultati ottenuti dal processo di annotazione.

In particolare, si osserva che l’unica annotazione effettuata rispetto all’ontologia **UMLS** ha prodotto un risultato completo, con l’annotazione di **tutti gli attributi previsti**.

La tabella  mostra che **solo una delle annotazioni effettuate tramite BioPortal** riesce a coprire **la totalità degli attributi**.

Per quanto riguarda le annotazioni eseguite con **MedCAT**, **tutte e quattro le tipologie** raggiungono una copertura completa (100% degli attributi), sia le **tre basate su SNOMED-CT** che quella basata su **UMLS**.

In particolare, l'annotazione con **UMLS**, che sfrutta la descrizione completa dell'attributo, restituisce **234 annotazioni distribuite su 41 attributi**, corrispondenti a una media di circa **5.71 annotazioni per attributo**.


In [15]:
# Totale distinti AttributoNomeCompleto
totale_distinti = annotazioni_both['AttributoNomeCompleto'].nunique()

print(totale_distinti)
# Group by con due aggregazioni: distinti e non distinti
risultato = (
    annotazioni_both
    .groupby(['TipoDiAnnotazione', 'Ontologies'])
    .agg(
        NumAttributiDistinti=('AttributoNomeCompleto', pd.Series.nunique),
        NumAnnotazioniTotali=('AttributoNomeCompleto', 'count')
    )
    .reset_index()
)

# Aggiunta percentuale rispetto al totale
risultato['PercSulTotale'] = (risultato['NumAttributiDistinti'] / totale_distinti * 100).round(2)

risultato

41


,TipoDiAnnotazione,Ontologies,NumAttributiDistinti,NumAnnotazioniTotali,PercSulTotale
0,BioportalAttributo,SNOMED-CT,26,31,63.41
1,BioportalAttributoNomeCompleto,SNOMED-CT,41,66,100.00
2,BioportalDescrizioneAttributo,SNOMED-CT,34,59,82.93
3,BioportalDescrizioneCompleta,SNOMED-CT,41,82,100.00
4,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,41,179,100.00
5,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,41,106,100.00
6,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,41,186,100.00
7,Medcat_UMLS_Df_Full_ANC,UMLS,41,234,100.00


## 3) DA METTERE NELLA TESI

 **Limitazione del Gold Standard per la Valutazione**

Per effettuare una valutazione corretta, è necessario considerare soltanto il gold standard relativo agli attributi effettivamente annotati dagli annotatori.  
Pertanto, il gold standard "generale" va ristretto a un **sottoinsieme** che contenga **solo gli attributi oggetto di valutazione**, ovvero **quelle coppie (ontologia, attributo)** che compaiono anche nel file delle annotazioni da valutare.  
Chiamiamo tale sottoinsieme *GS_both*.

In questo modo, il confronto sarà limitato alle sole informazioni effettivamente annotate, garantendo una valutazione coerente e significativa.


QUINDI NELLA TESI NON SI METTE IL CALCOLO DI GS_BOTH MA SOLO LA SUA DEFINIZIONE E QUANTI ELEMENTI HA, OVVERO LA TABELLA

In [16]:
# Totale distinti AttributoNomeCompleto
totale_distinti = GS_both['AttributoNomeCompleto'].nunique()
print(totale_distinti)


# Group by con due aggregazioni: distinti e non distinti
risultato = (
    GS_both
    .groupby(['TipoDiAnnotazione', 'Ontologies'])
    .agg(
        NumAttributiDistinti=('AttributoNomeCompleto', pd.Series.nunique),
        NumAnnotazioniTotali=('AttributoNomeCompleto', 'count')
    )
    .reset_index()
)

# Aggiunta percentuale rispetto al totale
risultato['PercSulTotale'] = (risultato['NumAttributiDistinti'] / totale_distinti * 100).round(2)

risultato

37


,TipoDiAnnotazione,Ontologies,NumAttributiDistinti,NumAnnotazioniTotali,PercSulTotale
0,MedicalGS,SNOMED-CT,33,52,89.19
1,MedicalGS,UMLS,37,82,100.00


In [17]:
# attributi

valid_attributes=['person-person_source_value', 'condition_occurrence-condition_status_source_value','location-location_id']

# Dizionario per salvare i risultati per ogni tipo di annotazione
df_results_per_type = {}

# Lista dei tipi di annotazione
tipi_di_annotazione = [
    'BioportalDescrizioneCompleta',
    'Medcat_SNOMED_Df_CS_1_ANC',
    'Medcat_SNOMED_Df_Full_ANC'
]

for tipo in tipi_di_annotazione:
    # Filtra solo le righe con il TipoDiAnnotazione corrente
    Xannotazioni_both = annotazioni_both[annotazioni_both['TipoDiAnnotazione'] == tipo]

    # Filtra solo gli attributi validi
    Xannotazioni_both = Xannotazioni_both[Xannotazioni_both['AttributoNomeCompleto'].isin(valid_attributes)]

    # Ottieni gli attributi unici rimasti
    attributi_filtered = Xannotazioni_both[['AttributoNomeCompleto']].drop_duplicates()

    # Crea i dizionari per GS e ANN solo per gli attributi filtrati
    GS_sets = GS_both.groupby('AttributoNomeCompleto')['Cui'].apply(set).reindex(attributi_filtered['AttributoNomeCompleto'], fill_value=set())
    ANN_sets = Xannotazioni_both.groupby('AttributoNomeCompleto')['Cui'].apply(set).reindex(attributi_filtered['AttributoNomeCompleto'], fill_value=set())

    # Creiamo il DataFrame per questo tipo di annotazione
    df_result_tipo = attributi_filtered.copy()
    df_result_tipo['GS_sets'] = df_result_tipo['AttributoNomeCompleto'].map(GS_sets)
    df_result_tipo['ANN_sets'] = df_result_tipo['AttributoNomeCompleto'].map(ANN_sets)

    # Salviamo il DataFrame nel dizionario con la chiave corrispondente al tipo di annotazione
    df_results_per_type[tipo] = df_result_tipo

# Ora puoi accedere ai risultati per ogni tipo di annotazione
df_results_per_type['BioportalDescrizioneCompleta']


,AttributoNomeCompleto,GS_sets,ANN_sets
34060,person-person_source_value,"{C0027361, C0449416, 125676002, 260753009, C15...","{125676002, 260753009}"
35147,condition_occurrence-condition_status_source_v...,"{64572001, C0012634, C0449438, C0449416, 26349...","{64572001, 125676002}"
35236,location-location_id,"{C0450429, 246267002}","{397635003, 246267002}"


ma gli insiemi non sono "confrontabili" in GS_set ci sono gli elementi C ....

non credo funzioni fatta così, vengono dei valori bassi

provo a rifare tutto

## 4) da mettere nella tesi


In definitiva, abbiamo due DataFrame distinti:

- Il primo *GS_both* contiene il gold standard
- Il secondo *annotazioni_both* contiene le annotazioni

Per semplicità, nel seguito ci riferiremo a questi due DataFrame semplicemente come *GoldStandard* e *Annotazioni*

In [18]:
GoldStandard=  GS_both
Annotazioni = annotazioni_both

## 5) da mettere nella tesi

mettere nella tesi seguente discorso e tabella con 4 elementi

Per calcolare l'insieme *ANN*, ovvero la lista dei codici che costituiscono l'annotazione  
di un certo elemento *AttributoNomeCompleto* rispetto a una certa *ontologia*, ottenuta da un certo *Annotator*,  
eseguiamo un **raggruppamento su tali campi**.

Questo ci permette di ottenere in uscita:

- la lista dei codici (*ANN*), costruita come l'insieme **ordinato** dei codici unici (`Cui`);
- il numero di elementi distinti che la compongono (*NumElementi*).

In [151]:
ANN_set = (
    Annotazioni
    .groupby(['TipoDiAnnotazione', 'Ontologies', 'Annotator', 'AttributoNomeCompleto'])
    .agg(
        NumElementi=('Cui', 'nunique'),  # count distinct
        ANN=('Cui', lambda x: set(x.unique()))
    )
    .reset_index()
)
ANN_set.sample(4)

,TipoDiAnnotazione,Ontologies,Annotator,AttributoNomeCompleto,NumElementi,ANN
149,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,MedCAT,condition_occurrence-condition_start_datetime,2,"{64572001, 125676002}"
147,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,MedCAT,condition_occurrence-condition_source_value,3,"{64572001, 125676002, 260753009}"
257,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,MedCAT,person-month_of_birth,4,"{258706009, 184099003, 125676002, 260753009}"
260,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,MedCAT,person-provider_id,2,"{125676002, 260753009}"


## 6) da mettere nella tesi



Applichiamo la stessa operazione anche al gold standard:  
raggruppiamo per i campi *Ontologies* e *AttributoNomeCompleto*  
per ottenere l'insieme *GS*.

In questo caso non è necessario includere il campo *TipoDiAnnotazione* nel `groupby`,  
poiché il suo valore serve unicamente a indicare che si tratta del gold standard.  
Raggruppare includendo o escludendo questa colonna produce lo stesso risultato.

In [20]:
GS_set = (
    GoldStandard
    .groupby(['TipoDiAnnotazione', 'Ontologies',  'AttributoNomeCompleto'])
    .agg(
        NumElementi=('Cui', 'nunique'),  # count distinct
        GS=('Cui', lambda x: set(x.unique()))
    )
    .reset_index()
)
GS_set.sample(4)

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,NumElementi,GS
54,MedicalGS,UMLS,location-state,2,"{C1442065, C1301808}"
48,MedicalGS,UMLS,location-address_1,1,{C1442065}
23,MedicalGS,SNOMED-CT,person-ethnicity_source_concept_id,1,{260753009}
55,MedicalGS,UMLS,location-zip,2,"{C1442065, C1521842}"


## 7) da mettere nella tesi


A questo punto, per ottenere in un unico DataFrame sia l'insieme *ANN* (annotazioni)  
che l'insieme *GS* (gold standard), è sufficiente effettuare una **join** tra i due dataset.

In particolare, utilizziamo una **full outer join** per assicurarci di considerare anche i casi  
in cui un elemento sia presente **solo nel gold standard** oppure **solo nel file delle annotazioni**.

L'output sarà un DataFrame che include **entrambi gli insiemi** per ogni combinazione rilevante  
di *ontologia* e *attributo*.

Includiamo queste operazioni in una funzione
che ha come input
sia i dataframe  *ANN* e  *GS* (gold standard),
sia il *TipoAnnotazione* che vogliamo valutare sia ovviamente l'*Ontologia* rispetto alla quale è stata fatta l'annotazone



In [21]:
def get_Annotazioni(FissaTipoAnnotazione, FissaOntologia,
                      ANN_set, GS_set):

    # Filtro annotazioni per tipo e ontologia
    ANN_set_Fixed = ANN_set[
        (ANN_set.TipoDiAnnotazione == FissaTipoAnnotazione) &
        (ANN_set.Ontologies == FissaOntologia)
    ]

    # Filtro gold standard per ontologia
    GS_set_Fixed = GS_set[
        GS_set.Ontologies == FissaOntologia
    ]

    # Full outer join
    merged = pd.merge(
        GS_set_Fixed[['Ontologies', 'AttributoNomeCompleto', 'GS']],
        ANN_set_Fixed[['Ontologies', 'AttributoNomeCompleto', 'TipoDiAnnotazione', 'ANN']],
        on=['Ontologies', 'AttributoNomeCompleto'],
        how='outer'
    )

    # Rinomina le colonne
    merged = merged.rename(columns={
        'GS': 'GS_set',
        'ANN': 'ANN_set'
    })

    # Riordina le colonne
    merged = merged[['TipoDiAnnotazione', 'Ontologies', 'AttributoNomeCompleto', 'GS_set', 'ANN_set']]

    return merged



Ad esempio

In [22]:
get_Annotazioni('Medcat_UMLS_Df_Full_ANC',
                'UMLS',
                ANN_set,
                GS_set
                ).sample(4)

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,GS_set,ANN_set
36,Medcat_UMLS_Df_Full_ANC,UMLS,person-provider_id,{C1138603},"{C0027361, C0449416}"
24,Medcat_UMLS_Df_Full_ANC,UMLS,person-care_site_id,NaN,"{C0027361, C0449416}"
34,Medcat_UMLS_Df_Full_ANC,UMLS,person-person_id,NaN,"{C0027361, C0449416}"
30,Medcat_UMLS_Df_Full_ANC,UMLS,person-gender_source_concept_id,"{C0178566, C0079399, C0449416}","{C0178566, C0079399, C0027361, C0449416}"


In [23]:
get_Annotazioni('BioportalDescrizioneCompleta',
                'SNOMED-CT',
                ANN_set,
                GS_set
                ).sort_values("AttributoNomeCompleto").head()

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,GS_set,ANN_set
0,BioportalDescrizioneCompleta,SNOMED-CT,condition_occurrence-condition_concept_id,{64572001},"{64572001, 125676002}"
1,BioportalDescrizioneCompleta,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 410671006}","{64572001, 125676002}"
2,BioportalDescrizioneCompleta,SNOMED-CT,condition_occurrence-condition_end_datetime,"{64572001, 397898000, 410671006}","{64572001, 125676002}"
3,BioportalDescrizioneCompleta,SNOMED-CT,condition_occurrence-condition_occurrence_id,"{64572001, 246454002}","{64572001, 125676002}"
4,BioportalDescrizioneCompleta,SNOMED-CT,condition_occurrence-condition_source_concept_id,"{64572001, 260753009}","{64572001, 125676002}"


In [24]:
get_Annotazioni('BioportalDescrizioneAttributo',
                'SNOMED-CT',
                ANN_set,
                GS_set
                ).sort_values("AttributoNomeCompleto").head()

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,GS_set,ANN_set
0,NaN,SNOMED-CT,condition_occurrence-condition_concept_id,{64572001},NaN
1,BioportalDescrizioneAttributo,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 410671006}",{410671006}
2,BioportalDescrizioneAttributo,SNOMED-CT,condition_occurrence-condition_end_datetime,"{64572001, 397898000, 410671006}",{410671006}
3,BioportalDescrizioneAttributo,SNOMED-CT,condition_occurrence-condition_occurrence_id,"{64572001, 246454002}","{272379006, 246454002}"
4,BioportalDescrizioneAttributo,SNOMED-CT,condition_occurrence-condition_source_concept_id,"{64572001, 260753009}",{260753009}


# Precision, Recall e Jaccard Similarity



Dato un attributo `AttributoNomeCompleto`, definiamo due insiemi:

- **Gold Standard (GS)**: insieme dei concetti ritenuti corretti per quell’attributo.
- **Annotazioni (ANN)**: insieme dei concetti predotti dall’annotator  per lo stesso attributo.


**Formula delle metriche**

- **Precision**:  
$
\text{Precision} = \frac{|GS \cap ANN|}{|ANN|}
$
Indica la frazione degli elementi predetti da ANN che sono effettivamente corretti.

- **Recall**:  
$
\text{Recall} = \frac{|GS \cap ANN|}{|GS|}
$
Indica la frazione degli elementi effettivamente presenti in GS che sono stati trovati da ANN.

- **Jaccard Similarity**:  
$
J(GS, ANN) = \frac{|GS \cap ANN|}{|GS \cup ANN|}
$
Misura la sovrapposizione tra GS e ANN rispetto all’unione di entrambi.

Per fare un esempio significativo di questi concetti, individuo tre attributi in cui le annotazioni variano con TipoDiAnnotazione;
ovviamente questi attributi devono anche avere il relativo gold standard

Il codice che segue serve per individuare questi tre attributi (rispetto a tre TipiDiAnnotazione) per i quali si hanno valori significativi


![immagine.png](attachment:ca920d53-6ec7-4111-8565-3de164851cb6.png)




In [25]:
# QUESTA PARTE SERVE SOLO PER TROVARE TRE attributi che hanno annotazioni per tutte e tre i tipi e quindi valori significativi
# Filtra le righe con tre tipi
filtered = annotazioni_both[annotazioni_both['TipoDiAnnotazione'].isin(['Medcat_SNOMED_Df_Full_ANC',  'Medcat_SNOMED_Df_CS_1_ANC', 'BioportalDescrizioneCompleta'])]
# Trova gli attributi che appaiono con entrambi i valori richiesti
common_attributes = filtered.groupby('AttributoNomeCompleto')['TipoDiAnnotazione'].nunique()
valid_attributes = common_attributes[common_attributes == 3].index.tolist()

#filtered=filtered[filtered['AttributoNomeCompleto'].isin(valid_attributes[:3])]
##### NON FUNZIONA BENE PER PRENDERE ESEMPI SIGNIFICATIVI CON DIVERSE ANNOTAZIONI
# dopo alcune prove li scelgo a mano
#filtered=filtered[filtered['AttributoNomeCompleto'].isin(['person-person_source_value', 'person-location_id', 'location-county'])]
filtered=filtered[filtered['AttributoNomeCompleto'].isin(['person-person_source_value',
'condition_occurrence-condition_status_source_value','location-location_id',
                                                         # 'person-location_id', 'location-county'
                                                         ])]


valid_attributes=['person-person_source_value', 'condition_occurrence-condition_status_source_value','location-location_id']
filtered[['Cui']].drop_duplicates()

,Cui
6374,125676002
7765,64572001
7772,263490005
7863,246267002
11388,260753009
13340,397635003


In [26]:
############## PER CONSIDERARE INVECE TUTTI GLI ATTRIBUTI

filtered = annotazioni_both
filtered[['Cui']].drop_duplicates()

,Cui
1281,125676002
1286,260753009
1304,263495000
1317,184099003
1332,258706009
1413,246267002
1415,397635003
2732,64572001
2947,263490005
3093,398070004


In [27]:
# Dizionario per salvare i risultati per ogni tipo di annotazione
df_results_per_type = {}

# Lista dei tipi di annotazione
tipi_di_annotazione = [
    'BioportalDescrizioneCompleta',
    'Medcat_SNOMED_Df_CS_1_ANC',
    'Medcat_SNOMED_Df_Full_ANC'
]

for tipo in tipi_di_annotazione:
    # Filtra solo le righe con il TipoDiAnnotazione corrente
    Xannotazioni_both = annotazioni_both[annotazioni_both['TipoDiAnnotazione'] == tipo]

    # Filtra solo gli attributi validi
    Xannotazioni_both = Xannotazioni_both[Xannotazioni_both['AttributoNomeCompleto'].isin(valid_attributes)]

    # Ottieni gli attributi unici rimasti
    attributi_filtered = Xannotazioni_both[['AttributoNomeCompleto']].drop_duplicates()

    # Crea i dizionari per GS e ANN solo per gli attributi filtrati
    GS_sets = GS_both.groupby('AttributoNomeCompleto')['Cui'].apply(set).reindex(attributi_filtered['AttributoNomeCompleto'], fill_value=set())
    ANN_sets = Xannotazioni_both.groupby('AttributoNomeCompleto')['Cui'].apply(set).reindex(attributi_filtered['AttributoNomeCompleto'], fill_value=set())

    # Creiamo il DataFrame per questo tipo di annotazione
    df_result_tipo = attributi_filtered.copy()
    df_result_tipo['GS_sets'] = df_result_tipo['AttributoNomeCompleto'].map(GS_sets)
    df_result_tipo['ANN_sets'] = df_result_tipo['AttributoNomeCompleto'].map(ANN_sets)

    # Salviamo il DataFrame nel dizionario con la chiave corrispondente al tipo di annotazione
    df_results_per_type[tipo] = df_result_tipo

# Ora puoi accedere ai risultati per ogni tipo di annotazione
df_results_per_type['BioportalDescrizioneCompleta']


,AttributoNomeCompleto,GS_sets,ANN_sets
34060,person-person_source_value,"{C0027361, C0449416, 125676002, 260753009, C15...","{125676002, 260753009}"
35147,condition_occurrence-condition_status_source_v...,"{64572001, C0012634, C0449438, C0449416, 26349...","{64572001, 125676002}"
35236,location-location_id,"{C0450429, 246267002}","{397635003, 246267002}"


In [28]:
import numpy as np

# Lista dei tipi di annotazione
tipi_di_annotazione = [
    'BioportalDescrizioneCompleta',
    'Medcat_SNOMED_Df_CS_1_ANC',
    'Medcat_SNOMED_Df_Full_ANC'
]

# Lista per raccogliere i DataFrame elaborati
df_list = []

for tipo in tipi_di_annotazione:
    X = df_results_per_type[tipo].copy()

    # Calcoliamo l'intersezione
    X['Intersezione'] = X.apply(lambda row: row['GS_sets'] & row['ANN_sets'], axis=1)

    # Calcoliamo la Jaccard Similarity
    X['Jaccard_Similarity'] = X.apply(
        lambda row: len(row['Intersezione']) / len(row['GS_sets'] | row['ANN_sets']) if row['GS_sets'] | row['ANN_sets'] else np.nan,
        axis=1
    )

    # Placeholder per il Weighted Overlap (sostituire con pesi reali se disponibili)
    X['Weighted_Overlap'] = X['Jaccard_Similarity']  # Se non hai pesi, puoi usare Jaccard come proxy

    # Calcoliamo Precision, Recall e F1-score
    X['Precision'] = X.apply(
        lambda row: len(row['Intersezione']) / len(row['ANN_sets']) if len(row['ANN_sets']) > 0 else np.nan,
        axis=1
    )

    X['Recall'] = X.apply(
        lambda row: len(row['Intersezione']) / len(row['GS_sets']) if len(row['GS_sets']) > 0 else np.nan,
        axis=1
    )

    X['F1_score'] = X.apply(
        lambda row: 2 * (row['Precision'] * row['Recall']) / (row['Precision'] + row['Recall'])
        if (row['Precision'] + row['Recall']) > 0 else np.nan,
        axis=1
    )

    # Aggiungiamo la colonna con il tipo di annotazione
    X['TipoDiAnnotazione'] = tipo

    # Aggiungiamo il DataFrame elaborato alla lista
    df_list.append(X)

# Concatenazione dei DataFrame in un unico DataFrame finale
df_final = pd.concat(df_list, ignore_index=True)

df_final.sort_values('F1_score', ascending=False).head(3)

,AttributoNomeCompleto,GS_sets,ANN_sets,Intersezione,Jaccard_Similarity,Weighted_Overlap,Precision,Recall,F1_score,TipoDiAnnotazione
0,person-person_source_value,"{C0027361, C0449416, 125676002, 260753009, C15...","{125676002, 260753009}","{125676002, 260753009}",0.400,0.400,1.00,0.400000,0.571429,BioportalDescrizioneCompleta
6,person-person_source_value,"{C0027361, C0449416, 125676002, 260753009, C15...","{125676002, 260753009}","{125676002, 260753009}",0.400,0.400,1.00,0.400000,0.571429,Medcat_SNOMED_Df_Full_ANC
7,condition_occurrence-condition_status_source_v...,"{64572001, C0012634, C0449438, C0449416, 26349...","{64572001, 263490005, 125676002, 260753009}","{64572001, 263490005, 260753009}",0.375,0.375,0.75,0.428571,0.545455,Medcat_SNOMED_Df_Full_ANC


In [29]:

# Selezioniamo i primi 5 attributi per ogni TipoDiAnnotazione ordinati per F1_score

top_5_per_tipo = (
    df_final.groupby('TipoDiAnnotazione')
    .apply(lambda group: group.nlargest(5, 'F1_score')[df_final.columns])
    .reset_index(drop=True)
)

# Visualizziamo il risultato
top_5_per_tipo.sort_values('AttributoNomeCompleto')
#

C:\Users\gabry\AppData\Local\Temp\ipykernel_319836\3442655723.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.nlargest(5, 'F1_score')[df_final.columns])


,AttributoNomeCompleto,GS_sets,ANN_sets,Intersezione,Jaccard_Similarity,Weighted_Overlap,Precision,Recall,F1_score,TipoDiAnnotazione
2,condition_occurrence-condition_status_source_v...,"{64572001, C0012634, C0449438, C0449416, 26349...","{64572001, 125676002}",{64572001},0.125000,0.125000,0.500000,0.142857,0.222222,BioportalDescrizioneCompleta
4,condition_occurrence-condition_status_source_v...,"{64572001, C0012634, C0449438, C0449416, 26349...","{64572001, 263490005, 125676002}","{64572001, 263490005}",0.250000,0.250000,0.666667,0.285714,0.400000,Medcat_SNOMED_Df_CS_1_ANC
7,condition_occurrence-condition_status_source_v...,"{64572001, C0012634, C0449438, C0449416, 26349...","{64572001, 263490005, 125676002, 260753009}","{64572001, 263490005, 260753009}",0.375000,0.375000,0.750000,0.428571,0.545455,Medcat_SNOMED_Df_Full_ANC
1,location-location_id,"{C0450429, 246267002}","{397635003, 246267002}",{246267002},0.333333,0.333333,0.500000,0.500000,0.500000,BioportalDescrizioneCompleta
3,location-location_id,"{C0450429, 246267002}","{125676002, 246267002}",{246267002},0.333333,0.333333,0.500000,0.500000,0.500000,Medcat_SNOMED_Df_CS_1_ANC
8,location-location_id,"{C0450429, 246267002}","{397635003, 125676002, 246267002}",{246267002},0.250000,0.250000,0.333333,0.500000,0.400000,Medcat_SNOMED_Df_Full_ANC
0,person-person_source_value,"{C0027361, C0449416, 125676002, 260753009, C15...","{125676002, 260753009}","{125676002, 260753009}",0.400000,0.400000,1.000000,0.400000,0.571429,BioportalDescrizioneCompleta
5,person-person_source_value,"{C0027361, C0449416, 125676002, 260753009, C15...",{125676002},{125676002},0.200000,0.200000,1.000000,0.200000,0.333333,Medcat_SNOMED_Df_CS_1_ANC
6,person-person_source_value,"{C0027361, C0449416, 125676002, 260753009, C15...","{125676002, 260753009}","{125676002, 260753009}",0.400000,0.400000,1.000000,0.400000,0.571429,Medcat_SNOMED_Df_Full_ANC


In [30]:
# Lista per raccogliere i risultati
results = []

# Iteriamo su ogni tipo di annotazione
for tipo in df_final['TipoDiAnnotazione'].unique():
    X = df_final[df_final['TipoDiAnnotazione'] == tipo]

    # Aggregazione Macro-average
    precision_macro = X['Precision'].mean()
    recall_macro = X['Recall'].mean()
    f1_macro = X['F1_score'].mean()

    # Aggregazione Micro-average
    total_tp = X['Intersezione'].apply(len).sum()
    total_ann = X['ANN_sets'].apply(len).sum()
    total_gs = X['GS_sets'].apply(len).sum()

    precision_micro = total_tp / total_ann if total_ann > 0 else np.nan
    recall_micro = total_tp / total_gs if total_gs > 0 else np.nan
    f1_micro = (
        2 * (precision_micro * recall_micro) / (precision_micro + recall_micro)
        if (precision_micro + recall_micro) > 0 else np.nan
    )

    # Aggiungiamo i risultati alla lista
    results.append([tipo, precision_macro, recall_macro, f1_macro, precision_micro, recall_micro, f1_micro])

# Creiamo il DataFrame finale
columns = ['TipoDiAnnotazione', 'Macro Precision', 'Macro Recall', 'Macro F1-score',
           'Micro Precision', 'Micro Recall', 'Micro F1-score']
df_metrics = pd.DataFrame(results, columns=columns)

# Visualizziamo il risultato
df_metrics

,TipoDiAnnotazione,Macro Precision,Macro Recall,Macro F1-score,Micro Precision,Micro Recall,Micro F1-score
0,BioportalDescrizioneCompleta,0.666667,0.347619,0.431217,0.666667,0.285714,0.400000
1,Medcat_SNOMED_Df_CS_1_ANC,0.722222,0.328571,0.411111,0.666667,0.285714,0.400000
2,Medcat_SNOMED_Df_Full_ANC,0.694444,0.442857,0.505628,0.666667,0.428571,0.521739


In [31]:
# indicativamente sembrano significativi
# la recall di MedCat Full è maggiore delle altre
# la recall di 1 è inferiore perchè prendi pochi elementi
# bioportal sembra avere valori confrontabili con MedCat full

In [32]:
#



###  Jaccard vs Precision/Recall/F1

Finora abbiamo usato Precision/Recall/F1

Se e a cosa serve la Jaccard


- **Jaccard** misura la sovrapposizione generale tra due insiemi.  
- **Precision, Recall e F1-score** valutano l’accuratezza dell’annotatore

vediamo la correlazione

**Caso in cui Recall = 1**

Quando la Recall è 1, significa che ANN ha trovato tutti gli elementi di GS.  
Quindi:
$
|GS \cap ANN| = |GS|
$

Ma sappiamo anche che:
$
|GS \cup ANN| = |GS| + |ANN| - |GS \cap ANN|
$

Sostituendo \( |GS \cap ANN| = |GS| \), otteniamo:
$
|GS \cup ANN| = |GS| + |ANN| - |GS| = |ANN|
$

Quindi:
$
Jaccard = \frac{|GS \cap ANN|}{|GS \cup ANN|} = \frac{|GS|}{|ANN|} = Precision
$

Quindi, se \( Recall = 1 \), la **Jaccard Similarity** è uguale alla **Precision** perché ANN contiene tutto GS, e quindi l’unione \( |GS \cup ANN| \) è semplicemente \( |ANN| \).  

**Caso in cui Recall < 1**

Quando **Recall è inferiore a 1**, significa che ANN ha perso alcuni elementi di GS.  
Quindi \( |GS \cap ANN| < |GS| \), e di conseguenza l’unione \( |GS \cup ANN| \) aumenta:
$
|GS \cup ANN| > |ANN|
$
Poiché il denominatore della **Jaccard** aumenta rispetto alla **Precision**, si ha che:
$
Jaccard < Precision
$


In [33]:
#dettagliare questa analisi rispetto alle diverse annotazioni --- QUESTO LO VERIFICHIAMO DOPO
# Aggiungiamo un annotatore fittizio per GS
GS['Annotator'] = 'Gold_Standard'

# Cambiamo il nome della colonna 'source' in 'Provenienza'
GS['Provenienza'] = 'GS'
annotazioni['Provenienza'] = 'annotazioni'
df_combined = pd.concat([GS, annotazioni])

# Raggruppiamo per Ontology e Annotator
ontology_annotator_counts = df_combined.groupby(['Ontologies', 'Annotator', 'Provenienza']).size().unstack(fill_value=0)

# Rinominiamo le colonne
ontology_annotator_counts.columns = ['Nleft', 'Nright']
ontology_annotator_counts['Nboth'] = ontology_annotator_counts.min(axis=1)

# Output finale
print(ontology_annotator_counts)

                          Nleft  Nright  Nboth
Ontologies Annotator                          
LOINC      Bioportal          0    7903      0
SNOMED-CT  Bioportal          0    4791      0
           Gold_Standard    127       0      0
           MedCAT             0   15607      0
UMLS       Gold_Standard    196       0      0
           MedCAT             0    9801      0


In [34]:
GS.columns

Index(['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
       'EntityID', 'PrettyName', 'Cui', 'Annotator', 'Provenienza'],
      dtype='object')

# Valutazione della Qualità degli Annotatori

## Metriche di valutazione

In [35]:
def evaluate_annotation(gs_df, ann_df, column):

    gs = set(gs_df[column].dropna())
    ann = set(ann_df[column].dropna())

    true_positives = len(gs & ann)  # Elementi corretti
    precision = true_positives / len(ann) if len(ann) > 0 else 0
    recall = true_positives / len(gs) if len(gs) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1-score": round(f1_score, 4)
    }

# Esempio di utilizzo


metrics = evaluate_annotation(GS, annotazioni, "AttributoNomeCompleto")
print(metrics)

{'Precision': 0.0576, 'Recall': 0.5541, 'F1-score': 0.1043}


In [36]:
def evaluate_macro_micro(gs_df, ann_df, columns):
    import pandas as pd

    def normalize(val):
        # converte stringhe separate da virgola in set di valori puliti e lowercase
        if pd.isna(val):
            return set()
        return set(v.strip().lower() for v in str(val).split(",") if v.strip())

    macro_precisions = []
    macro_recalls = []
    macro_f1s = []

    micro_tp = 0
    micro_ann_total = 0
    micro_gs_total = 0

    for col in columns:
        col_tp = 0
        col_ann_total = 0
        col_gs_total = 0

        for i in range(len(gs_df)):
            gs = normalize(gs_df.iloc[i][col])
            ann = normalize(ann_df.iloc[i][col])

            tp = len(gs & ann)
            ann_total = len(ann)
            gs_total = len(gs)

            col_tp += tp
            col_ann_total += ann_total
            col_gs_total += gs_total

        # Precision, recall, f1 per attributo (colonna)
        precision = col_tp / col_ann_total if col_ann_total > 0 else 0
        recall = col_tp / col_gs_total if col_gs_total > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        macro_precisions.append(precision)
        macro_recalls.append(recall)
        macro_f1s.append(f1)

        # accumulo per micro
        micro_tp += col_tp
        micro_ann_total += col_ann_total
        micro_gs_total += col_gs_total

    # Macro-average (media semplice sulle colonne)
    precision_macro = sum(macro_precisions) / len(columns) if columns else 0
    recall_macro = sum(macro_recalls) / len(columns) if columns else 0
    f1_macro = sum(macro_f1s) / len(columns) if columns else 0

    # Micro-average (totale veri positivi su totale annotazioni e ground truth)
    precision_micro = micro_tp / micro_ann_total if micro_ann_total > 0 else 0
    recall_micro = micro_tp / micro_gs_total if micro_gs_total > 0 else 0
    f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

    return {
        "Macro Precision": round(precision_macro, 4),
        "Macro Recall": round(recall_macro, 4),
        "Macro F1-score": round(f1_macro, 4),
        "Micro Precision": round(precision_micro, 4),
        "Micro Recall": round(recall_micro, 4),
        "Micro F1-score": round(f1_micro, 4)
    }




# DOMANDA

però in questo modo confronto tutti gli elementi del gs  con tutti gli elementi dell'annotazioni,
*indipendentemente dagli attributi ai quali sono associati*

vediamo cosa succede con i tre attributi di sopra

In [37]:
GS_filtered=GS[GS['AttributoNomeCompleto'].isin(['person-person_source_value',
'condition_occurrence-condition_status_source_value','location-location_id',
                                                         # 'person-location_id', 'location-county'
                                                         ])]

annotazioni_filtered=annotazioni[annotazioni['AttributoNomeCompleto'].isin(['person-person_source_value',
'condition_occurrence-condition_status_source_value','location-location_id',
                                                         # 'person-location_id', 'location-county'
                                                         ])]

In [38]:
evaluate_annotation(GS_filtered, annotazioni_filtered, "AttributoNomeCompleto")

{'Precision': 1.0, 'Recall': 1.0, 'F1-score': 1.0}

In [39]:
evaluate_macro_micro(GS_filtered, annotazioni_filtered, ["AttributoNomeCompleto"])

{'Macro Precision': 0.1875,
 'Macro Recall': 0.1875,
 'Macro F1-score': 0.1875,
 'Micro Precision': 0.1875,
 'Micro Recall': 0.1875,
 'Micro F1-score': 0.1875}

non tengo conto del confronto attributo per attributo (oltre al fatto di considerare tutti assieme tutti i tipi di annotazione)
quindi del fatto che che ad ogni attributo il GS associa un certo insieme di attributi e un tipo di annotazione un altro
come appunto in questo esempio di riferimento (ricordo che era riferito ad un particolare TipoDiAnnotazione)

![immagine.png](attachment:65ceaf74-adf2-4cef-b576-ecb2cb011822.png)



prova a rivedere tutto il discorso fatto fin qui, poi si prosegue con il resto


un'altra prova che si potrebbe fare: fare l'unione (o l'intersezione) delle annotazioni prodotte dai vari tipi di annotazione
e verificare come cambiano i risultati

### UNIONE E INTERSEZIONE TRA DATABASE MEDCAT E BIOPORTAL

In [40]:
annotazioni.columns

Index(['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
       'EntityID', 'PrettyName', 'Cui', 'Annotator', 'Provenienza'],
      dtype='object')

In [41]:
def filtra_annotazioni(df, Sistema):
    # Mappatura colonne
    col_A = 'AttributoNomeCompleto'
    col_VOC = 'Ontologies'
    col_C = 'Cui'
    col_i = 'EntityID'
    col_SISTEMA = 'Annotator'

    # Normalizza annotator in maiuscolo per confronto
    df = df.copy()
    df[col_SISTEMA] = df[col_SISTEMA].str.upper()

    if Sistema == 'M':
        return df[df[col_SISTEMA] == 'MEDCAT'].copy()

    elif Sistema == 'B':
        return df[df[col_SISTEMA] == 'BIOPORTAL'].copy()

    elif Sistema == 'B_AND_M':
        medcat_triples = set(df[df[col_SISTEMA] == 'MEDCAT'][[col_A, col_VOC, col_C]].apply(tuple, axis=1))
        bioportal_triples = set(df[df[col_SISTEMA] == 'BIOPORTAL'][[col_A, col_VOC, col_C]].apply(tuple, axis=1))
        comuni = medcat_triples & bioportal_triples

        df_comuni = df[df.apply(lambda row: (row[col_A], row[col_VOC], row[col_C]) in comuni, axis=1)].copy()
        df_comuni[col_SISTEMA] = 'MEDCAT_AND_BIOPORTAL'
        return df_comuni

    elif Sistema == 'B_OR_M':
        medcat = df[df[col_SISTEMA] == 'MEDCAT']
        bioportal = df[df[col_SISTEMA] == 'BIOPORTAL']

        medcat_triples = set(medcat[[col_A, col_VOC, col_C]].apply(tuple, axis=1))
        bioportal_triples = set(bioportal[[col_A, col_VOC, col_C]].apply(tuple, axis=1))
        comuni = medcat_triples & bioportal_triples

        comuni_df = df[df.apply(
            lambda row: (row[col_A], row[col_VOC], row[col_C]) in comuni and row[col_SISTEMA] in ['MEDCAT', 'BIOPORTAL'],
            axis=1
        )]

        nuovi = comuni_df.copy()
        nuovi[col_SISTEMA] = 'BIOPORTAL_OR_MEDCAT'

        df_filtrato = df[~df.apply(
            lambda row: (row[col_A], row[col_VOC], row[col_C]) in comuni and row[col_SISTEMA] in ['MEDCAT', 'BIOPORTAL'],
            axis=1
        )]

        df_finale = pd.concat([df_filtrato, nuovi], ignore_index=True).sort_values(by=[col_A, col_i]).reset_index(drop=True)
        return df_finale

    else:
        raise ValueError("Parametro Sistema non valido. Usa 'M', 'B', 'B_AND_M' o 'B_OR_M'.")


## 11 giugno

In [42]:
def get_Annotazioni(FissaTipoAnnotazione, FissaOntologia,
                    ANN_set, GS_set):

    # Filtro annotazioni per tipo e ontologia
    ANN_set_Fixed = ANN_set[
        ANN_set.TipoDiAnnotazione.str.startswith(FissaTipoAnnotazione) &
        (ANN_set.Ontologies == FissaOntologia)
    ]

    # Filtro gold standard per ontologia
    GS_set_Fixed = GS_set[
        GS_set.Ontologies == FissaOntologia
    ]

    # Full outer join
    merged = pd.merge(
        GS_set_Fixed[['Ontologies', 'AttributoNomeCompleto', 'GS']],
        ANN_set_Fixed[['Ontologies', 'AttributoNomeCompleto', 'TipoDiAnnotazione', 'ANN']],
        on=['Ontologies', 'AttributoNomeCompleto'],
        how='outer'
    )

    # Rinomina le colonne
    merged = merged.rename(columns={
        'GS': 'GS_set',
        'ANN': 'ANN_set'
    })

    # Riordina le colonne
    merged = merged[['TipoDiAnnotazione', 'Ontologies', 'AttributoNomeCompleto', 'GS_set', 'ANN_set']]

    return merged


In [43]:
medcat=get_Annotazioni('Medcat_SNOMED',
                'SNOMED-CT',
                ANN_set,
                GS_set)
#medcat.drop(columns='GS_set')

medcat


,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,GS_set,ANN_set
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,{64572001},"{64572001, 125676002}"
1,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,{64572001},"{64572001, 125676002}"
2,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,{64572001},"{64572001, 125676002}"
3,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 410671006}","{64572001, 125676002}"
4,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 410671006}","{64572001, 125676002}"
...,...,...,...,...,...
118,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,person-race_source_value,{260753009},{125676002}
119,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,person-race_source_value,{260753009},"{125676002, 260753009}"
120,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,person-year_of_birth,NaN,"{184099003, 125676002, 260753009}"
121,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,person-year_of_birth,NaN,"{184099003, 125676002}"


In [44]:
Bioportal =get_Annotazioni('Bioportal',
                'SNOMED-CT',
                ANN_set,
                GS_set)
#Bioportal.drop(columns='GS_set')
Bioportal = Bioportal[Bioportal['GS_set'].notnull()]
EsempioBioPortal=Bioportal.sample(10)
EsempioBioPortal

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,GS_set,ANN_set
6,BioportalAttributoNomeCompleto,SNOMED-CT,condition_occurrence-condition_end_datetime,"{64572001, 397898000, 410671006}",{246454002}
8,BioportalDescrizioneCompleta,SNOMED-CT,condition_occurrence-condition_end_datetime,"{64572001, 397898000, 410671006}","{64572001, 125676002}"
85,BioportalDescrizioneCompleta,SNOMED-CT,person-day_of_birth,{258703001},"{125676002, 260753009}"
131,BioportalAttributo,SNOMED-CT,person-race_source_concept_id,{260753009},{260753009}
13,BioportalAttributo,SNOMED-CT,condition_occurrence-condition_source_concept_id,"{64572001, 260753009}",{260753009}
69,BioportalDescrizioneCompleta,SNOMED-CT,location-location_id,{246267002},"{397635003, 246267002}"
96,BioportalDescrizioneCompleta,SNOMED-CT,person-ethnicity_source_value,{260753009},"{125676002, 260753009}"
60,BioportalDescrizioneCompleta,SNOMED-CT,location-address_2,{397635003},"{397635003, 246267002}"
31,BioportalAttributo,SNOMED-CT,condition_occurrence-condition_status_source_v...,"{64572001, 263490005, 260753009}","{263490005, 260753009}"
21,BioportalAttributo,SNOMED-CT,condition_occurrence-condition_start_date,"{64572001, 410671006}",{410671006}


In [45]:
def precision(gs, ann):
    return len(gs & ann) / len(ann) if ann else 0

def recall(gs, ann):
    return len(gs & ann) / len(gs) if gs else 0

def f1(p, r):
    return 2 * p * r / (p + r) if (p + r) > 0 else 0

# Calcolo riga per riga
EsempioBioPortal['Precision'] = EsempioBioPortal.apply(lambda row: precision(row['GS_set'], row['ANN_set']), axis=1)
EsempioBioPortal['Recall'] = EsempioBioPortal.apply(lambda row: recall(row['GS_set'], row['ANN_set']), axis=1)
EsempioBioPortal['F1'] = EsempioBioPortal.apply(lambda row: f1(row['Precision'], row['Recall']), axis=1)

#EsempioBioPortal[['AttributoNomeCompleto', 'Precision', 'Recall', 'F1']]
EsempioBioPortal

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,GS_set,ANN_set,Precision,Recall,F1
6,BioportalAttributoNomeCompleto,SNOMED-CT,condition_occurrence-condition_end_datetime,"{64572001, 397898000, 410671006}",{246454002},0.0,0.000000,0.000000
8,BioportalDescrizioneCompleta,SNOMED-CT,condition_occurrence-condition_end_datetime,"{64572001, 397898000, 410671006}","{64572001, 125676002}",0.5,0.333333,0.400000
85,BioportalDescrizioneCompleta,SNOMED-CT,person-day_of_birth,{258703001},"{125676002, 260753009}",0.0,0.000000,0.000000
131,BioportalAttributo,SNOMED-CT,person-race_source_concept_id,{260753009},{260753009},1.0,1.000000,1.000000
13,BioportalAttributo,SNOMED-CT,condition_occurrence-condition_source_concept_id,"{64572001, 260753009}",{260753009},1.0,0.500000,0.666667
69,BioportalDescrizioneCompleta,SNOMED-CT,location-location_id,{246267002},"{397635003, 246267002}",0.5,1.000000,0.666667
96,BioportalDescrizioneCompleta,SNOMED-CT,person-ethnicity_source_value,{260753009},"{125676002, 260753009}",0.5,1.000000,0.666667
60,BioportalDescrizioneCompleta,SNOMED-CT,location-address_2,{397635003},"{397635003, 246267002}",0.5,1.000000,0.666667
31,BioportalAttributo,SNOMED-CT,condition_occurrence-condition_status_source_v...,"{64572001, 263490005, 260753009}","{263490005, 260753009}",1.0,0.666667,0.800000
21,BioportalAttributo,SNOMED-CT,condition_occurrence-condition_start_date,"{64572001, 410671006}",{410671006},1.0,0.500000,0.666667


In [46]:
# Macro-average (media delle metriche)
precision_macro = EsempioBioPortal['Precision'].mean()
recall_macro = EsempioBioPortal['Recall'].mean()
f1_macro = EsempioBioPortal['F1'].mean()

# Stampa risultati
print("Macro-average metrics:")
print(f"Precision_macro: {precision_macro:.4f}")
print(f"Recall_macro:    {recall_macro:.4f}")
print(f"F1_macro:        {f1_macro:.4f}")

Macro-average metrics:
Precision_macro: 0.6000
Recall_macro:    0.6000
F1_macro:        0.5533


In [47]:

# Funzioni per metriche individuali
def precision(gs, ann):
    return len(gs & ann) / len(ann) if ann else 0

def recall(gs, ann):
    return len(gs & ann) / len(gs) if gs else 0

def f1(p, r):
    return 2 * p * r / (p + r) if (p + r) > 0 else 0

# Calcolo riga per riga
EsempioBioPortal['TP'] = EsempioBioPortal.apply(lambda row: len(row['GS_set'] & row['ANN_set']), axis=1)
EsempioBioPortal['GS_len'] = EsempioBioPortal['GS_set'].apply(len)
EsempioBioPortal['ANN_len'] = EsempioBioPortal['ANN_set'].apply(len)

EsempioBioPortal['Precision'] = EsempioBioPortal['TP'] / EsempioBioPortal['ANN_len']
EsempioBioPortal['Recall'] = EsempioBioPortal['TP'] / EsempioBioPortal['GS_len']
EsempioBioPortal['F1'] = EsempioBioPortal.apply(lambda row: f1(row['Precision'], row['Recall']), axis=1)

# Macro-average
precision_macro = EsempioBioPortal['Precision'].mean()
recall_macro = EsempioBioPortal['Recall'].mean()
f1_macro = EsempioBioPortal['F1'].mean()

# Micro-average
TP_total = EsempioBioPortal['TP'].sum()
GS_total = EsempioBioPortal['GS_len'].sum()
ANN_total = EsempioBioPortal['ANN_len'].sum()

precision_micro = TP_total / ANN_total if ANN_total > 0 else 0
recall_micro = TP_total / GS_total if GS_total > 0 else 0
f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

# Stampa risultati
print("Macro-average metrics:")
print(f"Precision_macro: {precision_macro:.4f}")
print(f"Recall_macro:    {recall_macro:.4f}")
print(f"F1_macro:        {f1_macro:.4f}\n")

print("Micro-average metrics:")
print(f"Precision_micro: {precision_micro:.4f}")
print(f"Recall_micro:    {recall_micro:.4f}")
print(f"F1_micro:        {f1_micro:.4f}")

Macro-average metrics:
Precision_macro: 0.6000
Recall_macro:    0.6000
F1_macro:        0.5533

Micro-average metrics:
Precision_micro: 0.5625
Recall_micro:    0.5000
F1_micro:        0.5294


In [94]:
# mettiamo tutto in una funzione
def calcola_Micro_Macro(Parametro):

    #FIX PER BIOPORTAL
     # Rimuovi righe con valori nulli
    Parametro = Parametro[Parametro['GS_set'].notnull() & Parametro['ANN_set'].notnull()]

    # Assicurati che siano set
    Parametro['GS_set'] = Parametro['GS_set'].apply(lambda x: set(x) if isinstance(x, (list, set)) else set())
    Parametro['ANN_set'] = Parametro['ANN_set'].apply(lambda x: set(x) if isinstance(x, (list, set)) else set())

    # Funzioni metriche
    def precision(gs, ann):
        return len(gs & ann) / len(ann) if ann else 0

    def recall(gs, ann):
        return len(gs & ann) / len(gs) if gs else 0

    def f1(p, r):
        return 2 * p * r / (p + r) if (p + r) > 0 else 0

    # Calcolo per ogni riga
    Parametro['TP'] = Parametro.apply(lambda row: len(row['GS_set'] & row['ANN_set']), axis=1)
    Parametro['GS_len'] = Parametro['GS_set'].apply(len)
    Parametro['ANN_len'] = Parametro['ANN_set'].apply(len)

    Parametro['Precision'] = Parametro['TP'] / Parametro['ANN_len']
    Parametro['Recall'] = Parametro['TP'] / Parametro['GS_len']
    Parametro['F1'] = Parametro.apply(lambda row: f1(row['Precision'], row['Recall']), axis=1)

    # Macro-average
    precision_macro = Parametro['Precision'].mean()
    recall_macro = Parametro['Recall'].mean()
    f1_macro = Parametro['F1'].mean()

    # Micro-average
    TP_total = Parametro['TP'].sum()
    GS_total = Parametro['GS_len'].sum()
    ANN_total = Parametro['ANN_len'].sum()

    precision_micro = TP_total / ANN_total if ANN_total > 0 else 0
    recall_micro = TP_total / GS_total if GS_total > 0 else 0
    f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

       # Creo DataFrame con una riga
    df_record = pd.DataFrame({
        "Precision_macro": [precision_macro],
        "Recall_macro": [recall_macro],
        "F1_macro": [f1_macro],
        "Precision_micro": [precision_micro],
        "Recall_micro": [recall_micro],
        "F1_micro": [f1_micro]
    })

    return df_record


In [97]:
def calcola_Micro_Macro(Parametro):
    # Rimuovi righe con valori nulli
    Parametro = Parametro[Parametro['GS_set'].notnull() & Parametro['ANN_set'].notnull()]

    # Assicurati che siano set
    Parametro['GS_set'] = Parametro['GS_set'].apply(lambda x: set(x) if isinstance(x, (list, set)) else set())
    Parametro['ANN_set'] = Parametro['ANN_set'].apply(lambda x: set(x) if isinstance(x, (list, set)) else set())

    # Funzioni metriche
    def precision(gs, ann):
        return len(gs & ann) / len(ann) if ann else 0

    def recall(gs, ann):
        return len(gs & ann) / len(gs) if gs else 0

    def f1(p, r):
        return 2 * p * r / (p + r) if (p + r) > 0 else 0

    # Calcolo per ogni riga
    Parametro['TP'] = Parametro.apply(lambda row: len(row['GS_set'] & row['ANN_set']), axis=1)
    Parametro['GS_len'] = Parametro['GS_set'].apply(len)
    Parametro['ANN_len'] = Parametro['ANN_set'].apply(len)

    Parametro['Precision'] = Parametro['TP'] / Parametro['ANN_len']
    Parametro['Recall'] = Parametro['TP'] / Parametro['GS_len']
    Parametro['F1'] = Parametro.apply(lambda row: f1(row['Precision'], row['Recall']), axis=1)

    # Macro
    precision_macro = Parametro['Precision'].mean()
    recall_macro = Parametro['Recall'].mean()
    f1_macro = Parametro['F1'].mean()

    # Micro
    TP_total = Parametro['TP'].sum()
    GS_total = Parametro['GS_len'].sum()
    ANN_total = Parametro['ANN_len'].sum()

    precision_micro = TP_total / ANN_total if ANN_total > 0 else 0
    recall_micro = TP_total / GS_total if GS_total > 0 else 0
    f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

    # Output
    return pd.DataFrame({
        "Precision_macro": [precision_macro],
        "Recall_macro": [recall_macro],
        "F1_macro": [f1_macro],
        "Precision_micro": [precision_micro],
        "Recall_micro": [recall_micro],
        "F1_micro": [f1_micro]
    })

In [49]:
calcola_Micro_Macro(EsempioBioPortal)

,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
0,0.6,0.6,0.553333,0.5625,0.5,0.529412


# 18 giugno: intersezione e Unione

vanno fatte riga per riga

In [50]:
Valutazione = pd.DataFrame(columns=['Precision_macro', 'Recall_macro', 'F1_macro', 'Precision_micro',
       'Recall_micro', 'F1_micro'])
Valutazione

,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro


In [51]:
BioportalA =get_Annotazioni('Bioportal',
                'SNOMED-CT',
                ANN_set,
                GS_set)
BioportalA = BioportalA[BioportalA['GS_set'].notnull()]
EsempioBioPortalA=BioportalA.sample(10)

In [52]:
Valutazione = pd.concat(
    [Valutazione, calcola_Micro_Macro(EsempioBioPortalA).rename(index={0: 'EsempioBioPortalA'})]
)
Valutazione

C:\Users\gabry\AppData\Local\Temp\ipykernel_319836\1486321603.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Valutazione = pd.concat(


,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
EsempioBioPortalA,0.45,0.666667,0.513333,0.444444,0.571429,0.5


In [53]:
MedCatB =get_Annotazioni('Medcat_SNOMED',
                'SNOMED-CT',
                ANN_set,
                GS_set)
MedCatB = MedCatB[MedCatB['GS_set'].notnull()]
EsempioMedCatB=MedCatB.sample(10)

Valutazione = pd.concat(
    [Valutazione, calcola_Micro_Macro(EsempioMedCatB).rename(index={0: 'EsempioMedCatB'})]
)
Valutazione



,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
EsempioBioPortalA,0.450,0.666667,0.513333,0.444444,0.571429,0.500000
EsempioMedCatB,0.375,0.600000,0.440000,0.333333,0.615385,0.432432


In [54]:
merged = pd.merge(
    EsempioBioPortalA, EsempioMedCatB,
    on=['Ontologies', 'AttributoNomeCompleto'],
    suffixes=('_1', '_2'),
    how='inner'
)
merged

,TipoDiAnnotazione_1,Ontologies,AttributoNomeCompleto,GS_set_1,ANN_set_1,TP_1,GS_len_1,ANN_len_1,Precision_1,Recall_1,F1_1,TipoDiAnnotazione_2,GS_set_2,ANN_set_2,TP_2,GS_len_2,ANN_len_2,Precision_2,Recall_2,F1_2
0,BioportalDescrizioneCompleta,SNOMED-CT,person-race_source_concept_id,{260753009},"{125676002, 260753009}",1,1,2,0.5,1.0,0.666667,Medcat_SNOMED_Df_CS_08_ANC,{260753009},"{125676002, 260753009}",1,1,2,0.5,1.0,0.666667


In [55]:


merged['ANN_setU'] = merged.apply(lambda row: row['ANN_set_1'].union(row['ANN_set_2']), axis=1)
merged['ANN_setI'] = merged.apply(lambda row: row['ANN_set_1'].intersection(row['ANN_set_2']), axis=1)

result = merged[['Ontologies', 'AttributoNomeCompleto', 'GS_set_1', 'ANN_setU', 'ANN_setI']].copy()
result = result.rename(columns={'GS_set_1': 'GS_set'})

result


,Ontologies,AttributoNomeCompleto,GS_set,ANN_setU,ANN_setI
0,SNOMED-CT,person-race_source_concept_id,{260753009},"{125676002, 260753009}","{125676002, 260753009}"


In [56]:
Intersezione = result[result['GS_set'].notnull()].copy()
Intersezione['TipoDiAnnotazione']='Intersezione'
Intersezione = Intersezione.drop(columns=['ANN_setU'])

Intersezione = Intersezione.rename(columns={'ANN_setI': 'ANN_set'})
Intersezione

,Ontologies,AttributoNomeCompleto,GS_set,ANN_set,TipoDiAnnotazione
0,SNOMED-CT,person-race_source_concept_id,{260753009},"{125676002, 260753009}",Intersezione


In [57]:
Valutazione = pd.concat(
    [Valutazione, calcola_Micro_Macro(Intersezione).rename(index={0: 'Intersezione'})]
)
Valutazione


,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
EsempioBioPortalA,0.450,0.666667,0.513333,0.444444,0.571429,0.500000
EsempioMedCatB,0.375,0.600000,0.440000,0.333333,0.615385,0.432432
Intersezione,0.500,1.000000,0.666667,0.500000,1.000000,0.666667


In [58]:
Unione = result[result['GS_set'].notnull()].copy()
Unione['TipoDiAnnotazione']='Unione'
Unione = Unione.drop(columns=['ANN_setI'])

Unione = Unione.rename(columns={'ANN_setU': 'ANN_set'})
Unione

,Ontologies,AttributoNomeCompleto,GS_set,ANN_set,TipoDiAnnotazione
0,SNOMED-CT,person-race_source_concept_id,{260753009},"{125676002, 260753009}",Unione


In [59]:
Valutazione = pd.concat(
    [Valutazione, calcola_Micro_Macro(Unione).rename(index={0: 'Unione'})]
)
Valutazione


,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
EsempioBioPortalA,0.450,0.666667,0.513333,0.444444,0.571429,0.500000
EsempioMedCatB,0.375,0.600000,0.440000,0.333333,0.615385,0.432432
Intersezione,0.500,1.000000,0.666667,0.500000,1.000000,0.666667
Unione,0.500,1.000000,0.666667,0.500000,1.000000,0.666667


In [60]:
def calcola_metriche_intersezione(Parametro):

    # Funzioni metriche
    def precision(gs, ann):
        return len(gs & ann) / len(ann) if ann else 0

    def recall(gs, ann):
        return len(gs & ann) / len(gs) if gs else 0

    def f1(p, r):
        return 2 * p * r / (p + r) if (p + r) > 0 else 0

    # Calcolo per ogni riga
    Parametro['TP'] = Parametro.apply(lambda row: len(row['GS_set'] & row['ANN_set']), axis=1)
    Parametro['GS_len'] = Parametro['GS_set'].apply(len)
    Parametro['ANN_len'] = Parametro['ANN_set'].apply(len)

    Parametro['Precision'] = Parametro['TP'] / Parametro['ANN_len']
    Parametro['Recall'] = Parametro['TP'] / Parametro['GS_len']
    Parametro['F1'] = Parametro.apply(lambda row: f1(row['Precision'], row['Recall']), axis=1)

    # Macro-average
    precision_macro = Parametro['Precision'].mean()
    recall_macro = Parametro['Recall'].mean()
    f1_macro = Parametro['F1'].mean()

    # Micro-average
    TP_total = Parametro['TP'].sum()
    GS_total = Parametro['GS_len'].sum()
    ANN_total = Parametro['ANN_len'].sum()

    precision_micro = TP_total / ANN_total if ANN_total > 0 else 0
    recall_micro = TP_total / GS_total if GS_total > 0 else 0
    f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

       # Creo DataFrame con una riga
    df_record = pd.DataFrame({
        "Precision_macro": [precision_macro],
        "Recall_macro": [recall_macro],
        "F1_macro": [f1_macro],
        "Precision_micro": [precision_micro],
        "Recall_micro": [recall_micro],
        "F1_micro": [f1_micro]
    })

    return df_record

In [61]:
evaluate_macro_micro(GS_set, Bioportal, ['AttributoNomeCompleto'])

{'Macro Precision': 0.0286,
 'Macro Recall': 0.0286,
 'Macro F1-score': 0.0286,
 'Micro Precision': 0.0286,
 'Micro Recall': 0.0286,
 'Micro F1-score': 0.0286}

In [62]:
GS_set.sample(4)

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,NumElementi,GS
39,MedicalGS,UMLS,condition_occurrence-condition_start_date,2,"{C0011008, C0012634}"
38,MedicalGS,UMLS,condition_occurrence-condition_source_value,3,"{C0012634, C1522609, C0449416}"
18,MedicalGS,SNOMED-CT,location-location_id,1,{246267002}
5,MedicalGS,SNOMED-CT,condition_occurrence-condition_source_value,2,"{64572001, 260753009}"


In [63]:
unione = (pd.concat([medcat, Bioportal]).drop(columns='GS_set'))
unione.head(5)

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,ANN_set
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,"{64572001, 125676002}"
1,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,"{64572001, 125676002}"
2,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,"{64572001, 125676002}"
3,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 125676002}"
4,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 125676002}"


In [64]:
GS_set

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,NumElementi,GS
0,MedicalGS,SNOMED-CT,condition_occurrence-condition_concept_id,1,{64572001}
1,MedicalGS,SNOMED-CT,condition_occurrence-condition_end_date,2,"{64572001, 410671006}"
2,MedicalGS,SNOMED-CT,condition_occurrence-condition_end_datetime,3,"{64572001, 397898000, 410671006}"
3,MedicalGS,SNOMED-CT,condition_occurrence-condition_occurrence_id,2,"{64572001, 246454002}"
4,MedicalGS,SNOMED-CT,condition_occurrence-condition_source_concept_id,2,"{64572001, 260753009}"
...,...,...,...,...,...
65,MedicalGS,UMLS,person-person_source_value,3,"{C1522609, C0027361, C0449416}"
66,MedicalGS,UMLS,person-provider_id,1,{C1138603}
67,MedicalGS,UMLS,person-race_concept_id,2,"{C0178566, C0034510}"
68,MedicalGS,UMLS,person-race_source_concept_id,3,"{C0178566, C0449416, C0034510}"


In [65]:
evaluate_annotation(GS_set, unione, 'AttributoNomeCompleto')

{'Precision': 0.9024, 'Recall': 1.0, 'F1-score': 0.9487}

In [66]:
evaluate_macro_micro(GS_set, unione, ['AttributoNomeCompleto'])

{'Macro Precision': 0.0429,
 'Macro Recall': 0.0429,
 'Macro F1-score': 0.0429,
 'Micro Precision': 0.0429,
 'Micro Recall': 0.0429,
 'Micro F1-score': 0.0429}

In [67]:
evaluate_macro_micro(GS_set, unione, ['TipoDiAnnotazione', 'Ontologies', 'AttributoNomeCompleto'])

{'Macro Precision': 0.1714,
 'Macro Recall': 0.1714,
 'Macro F1-score': 0.1714,
 'Micro Precision': 0.1714,
 'Micro Recall': 0.1714,
 'Micro F1-score': 0.1714}

In [68]:
intersezione = pd.merge(medcat,Bioportal, on=['AttributoNomeCompleto']).drop(columns=['GS_set_x','GS_set_y'])
intersezione.head(5)

,TipoDiAnnotazione_x,Ontologies_x,AttributoNomeCompleto,ANN_set_x,TipoDiAnnotazione_y,Ontologies_y,ANN_set_y
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,"{64572001, 125676002}",BioportalAttributoNomeCompleto,SNOMED-CT,{246454002}
1,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,"{64572001, 125676002}",BioportalDescrizioneCompleta,SNOMED-CT,"{64572001, 125676002}"
2,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,"{64572001, 125676002}",BioportalAttributoNomeCompleto,SNOMED-CT,{246454002}
3,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,"{64572001, 125676002}",BioportalDescrizioneCompleta,SNOMED-CT,"{64572001, 125676002}"
4,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,condition_occurrence-condition_concept_id,"{64572001, 125676002}",BioportalAttributoNomeCompleto,SNOMED-CT,{246454002}


In [69]:
evaluate_annotation(GS_set, intersezione, 'AttributoNomeCompleto')

{'Precision': 1.0, 'Recall': 0.8919, 'F1-score': 0.9429}

In [70]:
evaluate_macro_micro(GS_set, intersezione, ['AttributoNomeCompleto'])

{'Macro Precision': 0.0286,
 'Macro Recall': 0.0286,
 'Macro F1-score': 0.0286,
 'Micro Precision': 0.0286,
 'Micro Recall': 0.0286,
 'Micro F1-score': 0.0286}

## 20 GIUGNO

PARTE DI ESEMPIO PER DESCRIVERE IL PROCEDIMENTO CHE VERRÁ EFFETTUATO

In [71]:
BioportalA =get_Annotazioni('Bioportal',
                'SNOMED-CT',
                ANN_set,
                GS_set)
BioportalA = BioportalA[BioportalA['GS_set'].notnull()]
EsempioBioPortalA=BioportalA.sample(10)
BioportalA = BioportalA[BioportalA.TipoDiAnnotazione  =='BioportalAttributoNomeCompleto' ]

BioportalA

X1 = ['condition_occurrence-condition_end_date','condition_occurrence-stop_reason',
      'location-state','location-city', 'person-day_of_birth','person-ethnicity_source_value']

BioportalA[BioportalA['AttributoNomeCompleto'].isin(X1)]#.drop_duplicates()

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,GS_set,ANN_set
3,BioportalAttributoNomeCompleto,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 410671006}","{410671006, 246454002}"
44,BioportalAttributoNomeCompleto,SNOMED-CT,condition_occurrence-stop_reason,{64572001},{246454002}
61,BioportalAttributoNomeCompleto,SNOMED-CT,location-city,{397635003},{246267002}
75,BioportalAttributoNomeCompleto,SNOMED-CT,location-state,"{397635003, 398070004}","{398070004, 246267002}"
83,BioportalAttributoNomeCompleto,SNOMED-CT,person-day_of_birth,{258703001},"{258703001, 125676002}"
94,BioportalAttributoNomeCompleto,SNOMED-CT,person-ethnicity_source_value,{260753009},"{125676002, 260753009}"


In [72]:
MedCatB =get_Annotazioni('Medcat_SNOMED',
                'SNOMED-CT',
                ANN_set,
                GS_set)
MedCatB = MedCatB[MedCatB['GS_set'].notnull()]
EsempioMedCatB=MedCatB.sample(10)

MedCatB = MedCatB[MedCatB.TipoDiAnnotazione  =='Medcat_SNOMED_Df_Full_ANC' ]


MedCatB=MedCatB[MedCatB['AttributoNomeCompleto'].isin(X1)]#.drop_duplicates()
MedCatB

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,GS_set,ANN_set
5,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 410671006}","{64572001, 410671006, 125676002}"
41,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,condition_occurrence-stop_reason,{64572001},"{64572001, 125676002, 260753009}"
56,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,location-city,{397635003},"{397635003, 125676002, 246267002, 260753009}"
68,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,location-state,"{397635003, 398070004}","{397635003, 398070004, 125676002, 260753009, 2..."
77,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,person-day_of_birth,{258703001},"{258706009, 184099003, 125676002, 260753009}"
86,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,person-ethnicity_source_value,{260753009},"{125676002, 260753009}"


In [73]:
merged = pd.merge(
    BioportalA, MedCatB,
    on=['Ontologies', 'AttributoNomeCompleto'],
    suffixes=('_1', '_2'),
    how='inner'
)


merged['ANN_setU'] = merged.apply(lambda row: row['ANN_set_1'].union(row['ANN_set_2']), axis=1)
merged['ANN_setI'] = merged.apply(lambda row: row['ANN_set_1'].intersection(row['ANN_set_2']), axis=1)

result = merged[['Ontologies', 'AttributoNomeCompleto', 'GS_set_1', 'ANN_set_1','ANN_set_2', 'ANN_setU', 'ANN_setI']].copy()
result = result.rename(columns={'GS_set_1': 'GS_set'})
result = result.rename(columns={'ANN_set_1': 'ANN_set_BioportalANC'})
result = result.rename(columns={'ANN_set_2': 'ANN_set_Medcat_Full'})
result

,Ontologies,AttributoNomeCompleto,GS_set,ANN_set_BioportalANC,ANN_set_Medcat_Full,ANN_setU,ANN_setI
0,SNOMED-CT,condition_occurrence-condition_end_date,"{64572001, 410671006}","{410671006, 246454002}","{64572001, 410671006, 125676002}","{64572001, 410671006, 125676002, 246454002}",{410671006}
1,SNOMED-CT,condition_occurrence-stop_reason,{64572001},{246454002},"{64572001, 125676002, 260753009}","{64572001, 125676002, 246454002, 260753009}",{}
2,SNOMED-CT,location-city,{397635003},{246267002},"{397635003, 125676002, 246267002, 260753009}","{125676002, 260753009, 397635003, 246267002}",{246267002}
3,SNOMED-CT,location-state,"{397635003, 398070004}","{398070004, 246267002}","{397635003, 398070004, 125676002, 260753009, 2...","{125676002, 260753009, 397635003, 398070004, 2...","{398070004, 246267002}"
4,SNOMED-CT,person-day_of_birth,{258703001},"{258703001, 125676002}","{258706009, 184099003, 125676002, 260753009}","{258706009, 125676002, 260753009, 258703001, 1...",{125676002}
5,SNOMED-CT,person-ethnicity_source_value,{260753009},"{125676002, 260753009}","{125676002, 260753009}","{125676002, 260753009}","{125676002, 260753009}"


In [107]:
def calcola_Micro_Macro(Parametro):
    # Rimuove righe con valori nulli
    Parametro = Parametro[Parametro['GS_set'].notnull() & Parametro['ANN_set'].notnull()].copy()

    # Forza i tipi a set
    Parametro['GS_set'] = Parametro['GS_set'].apply(lambda x: set(x) if isinstance(x, (set, list)) else set())
    Parametro['ANN_set'] = Parametro['ANN_set'].apply(lambda x: set(x) if isinstance(x, (set, list)) else set())

    # Ora è sicuro fare l'intersezione
    Parametro['TP'] = Parametro.apply(
        lambda row: len(row['GS_set'].intersection(row['ANN_set'])), axis=1
    )

    # Calcoli ausiliari
    Parametro['GS_len'] = Parametro['GS_set'].apply(len)
    Parametro['ANN_len'] = Parametro['ANN_set'].apply(len)

    # Metriche riga per riga
    Parametro['Precision'] = Parametro['TP'] / Parametro['ANN_len'].replace(0, 1)
    Parametro['Recall'] = Parametro['TP'] / Parametro['GS_len'].replace(0, 1)
    Parametro['F1'] = Parametro.apply(
        lambda row: 2 * row['Precision'] * row['Recall'] / (row['Precision'] + row['Recall']) if (row['Precision'] + row['Recall']) > 0 else 0,
        axis=1
    )

    # Macro
    precision_macro = Parametro['Precision'].mean()
    recall_macro = Parametro['Recall'].mean()
    f1_macro = Parametro['F1'].mean()

    # Micro
    TP_total = Parametro['TP'].sum()
    GS_total = Parametro['GS_len'].sum()
    ANN_total = Parametro['ANN_len'].sum()

    precision_micro = TP_total / ANN_total if ANN_total > 0 else 0
    recall_micro = TP_total / GS_total if GS_total > 0 else 0
    f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

    return pd.DataFrame({
        "Precision_macro": [precision_macro],
        "Recall_macro": [recall_macro],
        "F1_macro": [f1_macro],
        "Precision_micro": [precision_micro],
        "Recall_micro": [recall_micro],
        "F1_micro": [f1_micro]
    })


In [138]:
Valutazione = pd.DataFrame(columns=['Precision_macro', 'Recall_macro', 'F1_macro', 'Precision_micro',
       'Recall_micro', 'F1_micro'])
Valutazione

,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro


In [139]:
lista_annotazioni = annotazioni["TipoDiAnnotazione"].unique().tolist()
lista_annotazioni

['Medcat_SNOMED_Df_CS_08_ANC',
 'Medcat_SNOMED_Df_CS_1_ANC',
 'Medcat_SNOMED_Df_Full_ANC',
 'Medcat_UMLS_Df_CS_08_ANC',
 'Medcat_UMLS_Df_CS_1_ANC',
 'Medcat_UMLS_Df_Full_ANC',
 'BioportalAttributoNomeCompleto',
 'BioportalDescrizioneCompleta',
 'BioportalDescrizioneAttributo',
 'BioportalAttributo']

In [140]:
lista_annotazioniMedcatSnomed = lista_annotazioni[:3]
lista_annotazioniMedcatSnomed

lista_annotazioniMedcatUMLS = lista_annotazioni[3:6]
lista_annotazioniMedcatUMLS

lista_annotazioniBioportal = lista_annotazioni[6:10]
lista_annotazioniBioportal

['BioportalAttributoNomeCompleto',
 'BioportalDescrizioneCompleta',
 'BioportalDescrizioneAttributo',
 'BioportalAttributo']

In [141]:
Bioportal = get_Annotazioni('Bioportal', 'UMLS', ANN_set, GS_set)
print(Bioportal['GS_set'].apply(type).value_counts())
print(Bioportal['ANN_set'].apply(type).value_counts())

GS_set
<class 'set'>    37
Name: count, dtype: int64
ANN_set
<class 'float'>    37
Name: count, dtype: int64


In [142]:
Bioportal ={}
for x in lista_annotazioniBioportal:
    Bioportal[x] = get_Annotazioni('Bioportal',
                                 'SNOMED-CT',
                                 ANN_set,
                                 GS_set)

    Bioportal[x] = Bioportal[x][Bioportal[x]['GS_set'].notnull()]
    Bioportal[x] = Bioportal[x][Bioportal[x]['TipoDiAnnotazione'] == x]

    Valutazione = pd.concat(
        [Valutazione, calcola_Micro_Macro(Bioportal[x]).rename(index={0: x})]
    )
Valutazione


C:\Users\gabry\AppData\Local\Temp\ipykernel_319836\2005435896.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Valutazione = pd.concat(


,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
BioportalAttributoNomeCompleto,0.484848,0.641414,0.533333,0.526316,0.588235,0.555556
BioportalDescrizioneCompleta,0.424242,0.590909,0.475758,0.424242,0.549020,0.478632
BioportalDescrizioneAttributo,0.676786,0.726190,0.636905,0.588235,0.666667,0.625000
BioportalAttributo,1.000000,0.840278,0.894444,1.000000,0.783784,0.878788


In [135]:
Bioportal

{'BioportalAttributoNomeCompleto':                   TipoDiAnnotazione Ontologies  \
 0    BioportalAttributoNomeCompleto  SNOMED-CT   
 3    BioportalAttributoNomeCompleto  SNOMED-CT   
 6    BioportalAttributoNomeCompleto  SNOMED-CT   
 10   BioportalAttributoNomeCompleto  SNOMED-CT   
 14   BioportalAttributoNomeCompleto  SNOMED-CT   
 18   BioportalAttributoNomeCompleto  SNOMED-CT   
 22   BioportalAttributoNomeCompleto  SNOMED-CT   
 25   BioportalAttributoNomeCompleto  SNOMED-CT   
 29   BioportalAttributoNomeCompleto  SNOMED-CT   
 32   BioportalAttributoNomeCompleto  SNOMED-CT   
 35   BioportalAttributoNomeCompleto  SNOMED-CT   
 44   BioportalAttributoNomeCompleto  SNOMED-CT   
 47   BioportalAttributoNomeCompleto  SNOMED-CT   
 50   BioportalAttributoNomeCompleto  SNOMED-CT   
 54   BioportalAttributoNomeCompleto  SNOMED-CT   
 58   BioportalAttributoNomeCompleto  SNOMED-CT   
 61   BioportalAttributoNomeCompleto  SNOMED-CT   
 64   BioportalAttributoNomeCompleto  SNOMED-CT 

In [143]:
Medcat = {}
for x in lista_annotazioniMedcatSnomed:
    Medcat[x] = get_Annotazioni('Medcat',
                                 'SNOMED-CT',
                                 ANN_set,
                                 GS_set)

    Medcat[x] = Medcat[x][Medcat[x]['GS_set'].notnull()]
    Medcat[x] = Medcat[x][Medcat[x]['TipoDiAnnotazione'] == x]

    Valutazione = pd.concat(
        [Valutazione, calcola_Micro_Macro(Medcat[x]).rename(index={0: x})]
    )
Valutazione

,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
BioportalAttributoNomeCompleto,0.484848,0.641414,0.533333,0.526316,0.588235,0.555556
BioportalDescrizioneCompleta,0.424242,0.590909,0.475758,0.424242,0.549020,0.478632
BioportalDescrizioneAttributo,0.676786,0.726190,0.636905,0.588235,0.666667,0.625000
BioportalAttributo,1.000000,0.840278,0.894444,1.000000,0.783784,0.878788
Medcat_SNOMED_Df_CS_08_ANC,0.426263,0.792929,0.525108,0.410526,0.764706,0.534247
Medcat_SNOMED_Df_CS_1_ANC,0.318182,0.373737,0.329293,0.333333,0.411765,0.368421
Medcat_SNOMED_Df_Full_ANC,0.436364,0.833333,0.548341,0.424242,0.823529,0.560000


In [144]:
for x in Bioportal:
    for y in Medcat:
        # Estraggo i DataFrame corrispondenti
        BioportalDF = Bioportal[x]
        MedcatDF = Medcat[y]

        # Faccio il merge sui campi chiave
        merged = pd.merge(
            BioportalDF, MedcatDF,
            on=['Ontologies', 'AttributoNomeCompleto'],
            suffixes=('_1', '_2'),
            how='inner'
        )

        # Assicurati che ANN_set siano set
        merged['ANN_set_1'] = merged['ANN_set_1'].apply(set)
        merged['ANN_set_2'] = merged['ANN_set_2'].apply(set)

        # Creo unione e intersezione tra gli insiemi
        merged['ANN_setU'] = merged.apply(lambda row: row['ANN_set_1'].union(row['ANN_set_2']), axis=1)
        merged['ANN_setI'] = merged.apply(lambda row: row['ANN_set_1'].intersection(row['ANN_set_2']), axis=1)

        # Seleziono e rinomino colonne
        result = merged[['Ontologies', 'AttributoNomeCompleto', 'GS_set_1', 'ANN_setU', 'ANN_setI']].copy()
        result = result.rename(columns={'GS_set_1': 'GS_set'})

        # --- Calcolo sulle INTERSEZIONI ---
        Tipo_intersezione = f'Intersezione_{y}_{x}'
        Intersezione = result[result['GS_set'].notnull()].copy()
        Intersezione['TipoDiAnnotazione'] = Tipo_intersezione
        Intersezione = Intersezione.drop(columns=['ANN_setU'])
        Intersezione = Intersezione.rename(columns={'ANN_setI': 'ANN_set'})

        Valutazione = pd.concat(
            [Valutazione, calcola_Micro_Macro(Intersezione).rename(index={0: Tipo_intersezione})]
        )

# Mostro Valutazione aggiornata
Valutazione



,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
BioportalAttributoNomeCompleto,0.484848,0.641414,0.533333,0.526316,0.588235,0.555556
BioportalDescrizioneCompleta,0.424242,0.590909,0.475758,0.424242,0.549020,0.478632
BioportalDescrizioneAttributo,0.676786,0.726190,0.636905,0.588235,0.666667,0.625000
BioportalAttributo,1.000000,0.840278,0.894444,1.000000,0.783784,0.878788
Medcat_SNOMED_Df_CS_08_ANC,0.426263,0.792929,0.525108,0.410526,0.764706,0.534247
Medcat_SNOMED_Df_CS_1_ANC,0.318182,0.373737,0.329293,0.333333,0.411765,0.368421
Medcat_SNOMED_Df_Full_ANC,0.436364,0.833333,0.548341,0.424242,0.823529,0.560000
Intersezione_Medcat_SNOMED_Df_CS_08_ANC_BioportalAttributoNomeCompleto,0.373737,0.489899,0.401010,0.589744,0.450980,0.511111
Intersezione_Medcat_SNOMED_Df_CS_1_ANC_BioportalAttributoNomeCompleto,0.196970,0.176768,0.171717,0.360000,0.176471,0.236842
Intersezione_Medcat_SNOMED_Df_Full_ANC_BioportalAttributoNomeCompleto,0.434343,0.520202,0.441414,0.609756,0.490196,0.543478


In [145]:
for x in Bioportal:
    for y in Medcat:
        # Estraggo i DataFrame corrispondenti
        BioportalDF = Bioportal[x]
        MedcatDF = Medcat[y]

        # Faccio il merge sui campi chiave
        merged = pd.merge(
            BioportalDF, MedcatDF,
            on=['Ontologies', 'AttributoNomeCompleto'],
            suffixes=('_1', '_2'),
            how='inner'
        )

        # Assicurati che ANN_set siano set
        merged['ANN_set_1'] = merged['ANN_set_1'].apply(set)
        merged['ANN_set_2'] = merged['ANN_set_2'].apply(set)

        # Creo unione e intersezione tra gli insiemi
        merged['ANN_setU'] = merged.apply(lambda row: row['ANN_set_1'].union(row['ANN_set_2']), axis=1)
        merged['ANN_setI'] = merged.apply(lambda row: row['ANN_set_1'].intersection(row['ANN_set_2']), axis=1)

        # Seleziono e rinomino colonne
        result = merged[['Ontologies', 'AttributoNomeCompleto', 'GS_set_1', 'ANN_setU', 'ANN_setI']].copy()
        result = result.rename(columns={'GS_set_1': 'GS_set'})

        # --- Calcolo sulle UNIONI ---
        Tipo_unione = f'Unione_{y}_{x}'
        Unione = result[result['GS_set'].notnull()].copy()
        Unione['TipoDiAnnotazione'] = Tipo_unione
        Unione = Unione.drop(columns=['ANN_setI'])
        Unione = Unione.rename(columns={'ANN_setU': 'ANN_set'})

        Valutazione = pd.concat(
            [Valutazione, calcola_Micro_Macro(Unione).rename(index={0: Tipo_unione})]
        )

# Mostro Valutazione aggiornata
Valutazione



,Precision_macro,Recall_macro,F1_macro,Precision_micro,Recall_micro,F1_micro
BioportalAttributoNomeCompleto,0.484848,0.641414,0.533333,0.526316,0.588235,0.555556
BioportalDescrizioneCompleta,0.424242,0.590909,0.475758,0.424242,0.549020,0.478632
BioportalDescrizioneAttributo,0.676786,0.726190,0.636905,0.588235,0.666667,0.625000
BioportalAttributo,1.000000,0.840278,0.894444,1.000000,0.783784,0.878788
Medcat_SNOMED_Df_CS_08_ANC,0.426263,0.792929,0.525108,0.410526,0.764706,0.534247
Medcat_SNOMED_Df_CS_1_ANC,0.318182,0.373737,0.329293,0.333333,0.411765,0.368421
Medcat_SNOMED_Df_Full_ANC,0.436364,0.833333,0.548341,0.424242,0.823529,0.560000
Intersezione_Medcat_SNOMED_Df_CS_08_ANC_BioportalAttributoNomeCompleto,0.373737,0.489899,0.401010,0.589744,0.450980,0.511111
Intersezione_Medcat_SNOMED_Df_CS_1_ANC_BioportalAttributoNomeCompleto,0.196970,0.176768,0.171717,0.360000,0.176471,0.236842
Intersezione_Medcat_SNOMED_Df_Full_ANC_BioportalAttributoNomeCompleto,0.434343,0.520202,0.441414,0.609756,0.490196,0.543478


sezione umls

In [146]:
ANN_set

,TipoDiAnnotazione,Ontologies,Annotator,AttributoNomeCompleto,NumElementi,ANN
0,BioportalAttributo,SNOMED-CT,Bioportal,condition_occurrence-condition_end_date,1,{410671006}
1,BioportalAttributo,SNOMED-CT,Bioportal,condition_occurrence-condition_occurrence_id,1,{246454002}
2,BioportalAttributo,SNOMED-CT,Bioportal,condition_occurrence-condition_source_concept_id,1,{260753009}
3,BioportalAttributo,SNOMED-CT,Bioportal,condition_occurrence-condition_source_value,1,{260753009}
4,BioportalAttributo,SNOMED-CT,Bioportal,condition_occurrence-condition_start_date,1,{410671006}
...,...,...,...,...,...,...
301,Medcat_UMLS_Df_Full_ANC,UMLS,MedCAT,person-provider_id,2,"{C0027361, C0449416}"
302,Medcat_UMLS_Df_Full_ANC,UMLS,MedCAT,person-race_concept_id,4,"{C0178566, C0034510, C0027361, C0449416}"
303,Medcat_UMLS_Df_Full_ANC,UMLS,MedCAT,person-race_source_concept_id,2,"{C0027361, C0449416}"
304,Medcat_UMLS_Df_Full_ANC,UMLS,MedCAT,person-race_source_value,5,"{C0027361, C0034510, C0449416, C0178566, C1522..."


In [148]:
GS_set

,TipoDiAnnotazione,Ontologies,AttributoNomeCompleto,NumElementi,GS
0,MedicalGS,SNOMED-CT,condition_occurrence-condition_concept_id,1,{64572001}
1,MedicalGS,SNOMED-CT,condition_occurrence-condition_end_date,2,"{64572001, 410671006}"
2,MedicalGS,SNOMED-CT,condition_occurrence-condition_end_datetime,3,"{64572001, 397898000, 410671006}"
3,MedicalGS,SNOMED-CT,condition_occurrence-condition_occurrence_id,2,"{64572001, 246454002}"
4,MedicalGS,SNOMED-CT,condition_occurrence-condition_source_concept_id,2,"{64572001, 260753009}"
...,...,...,...,...,...
65,MedicalGS,UMLS,person-person_source_value,3,"{C1522609, C0027361, C0449416}"
66,MedicalGS,UMLS,person-provider_id,1,{C1138603}
67,MedicalGS,UMLS,person-race_concept_id,2,"{C0178566, C0034510}"
68,MedicalGS,UMLS,person-race_source_concept_id,3,"{C0178566, C0449416, C0034510}"


In [116]:
for x in lista_annotazioniBioportal:
    Bioportal = get_Annotazioni('Bioportal',
                                 'UMLS',
                                 ANN_set,
                                 GS_set)

    Bioportal = Bioportal[Bioportal['GS_set'].notnull()]
    Bioportal = Bioportal[Bioportal['TipoDiAnnotazione'] == x]

    Valutazione = pd.concat(
        [Valutazione, calcola_Micro_Macro(Bioportal).rename(index={0: x})]
    )
Valutazione

ValueError: Cannot set a DataFrame with multiple columns to the single column TP

In [149]:

Medcat = {}
for x in lista_annotazioniMedcatUMLS:
    Medcat[x] = get_Annotazioni('Medcat',
                                 'SNOMED-CT',
                                 ANN_set,
                                 GS_set)

    Medcat[x] = Medcat[x][Medcat[x]['GS_set'].notnull()]
    Medcat[x] = Medcat[x][Medcat[x]['TipoDiAnnotazione'] == x]

    Valutazione = pd.concat(
        [Valutazione, calcola_Micro_Macro(Medcat[x]).rename(index={0: x})]
    )
Valutazione

ValueError: Cannot set a DataFrame with multiple columns to the single column TP

# 18 giugno


con
evaluate_macro_micro(GS_set, intersezione, ['AttributoNomeCompleto'])

ottengo




{'Macro Precision': 0.0286,
 'Macro Recall': 0.0286,
 'Macro F1-score': 0.0286,
 'Micro Precision': 0.0286,
 'Micro Recall': 0.0286,
 'Micro F1-score': 0.0286}


 QUESTI VALORI INDICANO CHE PROBABILMENTE IL CALCOLO CONTINUA AD ESSERE SBAGLIATO!



In [135]:
intersezione.columns

Index(['TipoDiAnnotazione_x', 'Ontologies_x', 'AttributoNomeCompleto',
       'ANN_set_x', 'TipoDiAnnotazione_y', 'Ontologies_y', 'ANN_set_y'],
      dtype='object')

In [136]:
evaluate_annotation(GS, unione, 'AttributoNomeCompleto')

{'Precision': 1.0, 'Recall': 0.5541, 'F1-score': 0.713}

In [137]:
evaluate_macro_micro(GS, unione, ['AttributoNomeCompleto'])

IndexError: single positional indexer is out-of-bounds

In [ ]:
evaluate_macro_micro(GS, unione, ['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
       'EntityID', 'PrettyName', 'Cui', 'Annotator', 'Provenienza'])

{'Macro Precision': 0.1629,
 'Macro Recall': 0.1631,
 'Macro F1-score': 0.163,
 'Micro Precision': 0.1618,
 'Micro Recall': 0.1631,
 'Micro F1-score': 0.1624}

analizzando le due valutazioni, si riscontra come ci sia un aumento della precision e dell' f1 score e recall uguale

In [ ]:
intersezione = filtra_annotazioni(annotazioni, 'B_OR_M')
intersezione.head(5)

,TipoDiAnnotazione,Ontologies,Source,AttributoNomeCompleto,EntityID,PrettyName,Cui,Annotator,Provenienza
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,admissions-admission_location,0,Hospital admission,32485007,MEDCAT,annotazioni
1,Medcat_SNOMED_Df_CS_1_ANC,SNOMED-CT,Mimic,admissions-admission_location,0,Hospital admission,32485007,MEDCAT,annotazioni
2,Medcat_SNOMED_Df_Full_ANC,SNOMED-CT,Mimic,admissions-admission_location,0,Hospital admission,32485007,MEDCAT,annotazioni
3,Medcat_UMLS_Df_Full_ANC,UMLS,Mimic,admissions-admission_location,0,"Admission, Patient",C0030673,MEDCAT,annotazioni
4,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Mimic,admissions-admission_location,1,"Table, device",86407004,MEDCAT,annotazioni


In [ ]:
evaluate_annotation(GS, intersezione, 'AttributoNomeCompleto')

{'Precision': 0.0576, 'Recall': 0.5541, 'F1-score': 0.1043}

In [ ]:
evaluate_macro_micro(GS, intersezione, ['AttributoNomeCompleto'])

{'Macro Precision': 0.0,
 'Macro Recall': 0.0,
 'Macro F1-score': 0.0,
 'Micro Precision': 0.0,
 'Micro Recall': 0.0,
 'Micro F1-score': 0}

In [ ]:
evaluate_macro_micro(GS, intersezione, ['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
       'EntityID', 'PrettyName', 'Cui', 'Annotator', 'Provenienza'])

{'Macro Precision': 0.0556,
 'Macro Recall': 0.0557,
 'Macro F1-score': 0.0557,
 'Micro Precision': 0.0549,
 'Micro Recall': 0.0557,
 'Micro F1-score': 0.0553}

## Metodi di confronto

In [ ]:
def metodi_confronto(gs_df, ann_df, column):

    gs = set(gs_df[column].dropna())
    ann = set(ann_df[column].dropna())

    intersection = gs & ann  # Concetti comuni

    # Jaccard Similarity
    jaccard = len(intersection) / len(gs | ann) if len(gs | ann) > 0 else 0

    # Weighted Overlap
    weighted_overlap = sum(1 for concept in intersection) / sum(1 for concept in gs) if len(gs) > 0 else 0

    return {
        "Intersection": len(intersection),
        "Jaccard Similarity": round(jaccard, 4),
        "Weighted Overlap": round(weighted_overlap, 4),
        "Valori dell'intersezione": intersection
    }



metrics = metodi_confronto(GS, annotazioni, "AttributoNomeCompleto")
print(metrics)

{'Jaccard Similarity': 0.055, 'Weighted Overlap': 0.5541, 'Intersection': 41, "Valori dell'intersezione": {'person-gender_source_concept_id', 'condition_occurrence-provider_id', 'condition_occurrence-condition_source_concept_id', 'person-ethnicity_source_value', 'condition_occurrence-condition_type_concept_id', 'location-zip', 'location-address_1', 'condition_occurrence-condition_status_source_value', 'person-year_of_birth', 'location-address_2', 'condition_occurrence-stop_reason', 'person-race_source_concept_id', 'person-gender_source_value', 'condition_occurrence-person_id', 'location-county', 'person-person_id', 'condition_occurrence-condition_concept_id', 'location-city', 'person-ethnicity_concept_id', 'person-month_of_birth', 'location-location_id', 'location-state', 'location-location_source_value', 'condition_occurrence-condition_occurrence_id', 'person-ethnicity_source_concept_id', 'condition_occurrence-visit_detail_id', 'person-race_source_value', 'condition_occurrence-conditi

In [ ]:
gs_df = GS.assign(source='GS')
# Calcoliamo la frequenza d'uso dei concetti nel Gold Standard
gs_freq = gs_df['AttributoNomeCompleto'].value_counts()


# Mappiamo la frequenza come peso nei concetti di GS
gs_df['Peso'] = gs_df['AttributoNomeCompleto'].map(gs_freq)


# Uniamo i due DataFrame per ottenere i concetti comuni
merged_df = pd.merge(gs_df, annotazioni, on='AttributoNomeCompleto', how='inner')

# Calcoliamo l'overlap ponderato sommando i pesi dei concetti comuni
overlap_ponderato = (merged_df['Peso'] * gs_freq[merged_df['AttributoNomeCompleto']]).sum()

print(f'Overlap Ponderato: {overlap_ponderato}')


Overlap Ponderato: 0.0


NON CAPISCO PERCHÈ L'OVERLAP PONDERATO SIA UGUALE A 0

In [ ]:
#Troviamo l'intersezione tra Gold Standard e Annotazioni
common_concepts = set(GS['AttributoNomeCompleto']).intersection(set(annotazioni['AttributoNomeCompleto']))

# 3. Calcoliamo la percentuale di concetti del Gold Standard presenti nelle Annotazioni
total_concepts_gs = len(GS)
correctly_annotated = len(common_concepts)

# Percentuale di concetti correttamente annotati
annotation_accuracy = (correctly_annotated / total_concepts_gs) * 100

# 4. Valutiamo la bontà degli annotatori
print(f'Concetti totali nel Gold Standard: {total_concepts_gs}')
print(f'Concetti correttamente annotati: {correctly_annotated}')
print(f'Accuratezza delle annotazioni: {annotation_accuracy:.2f}%')

Concetti totali nel Gold Standard: 323
Concetti correttamente annotati: 41
Accuratezza delle annotazioni: 12.69%


## Calcolo metriche su più attributi

In [ ]:
def calculate_metrics(gs_df, ann_df):
    # Inizializziamo un dizionario per memorizzare i risultati per ogni concetto
    metrics = {'AttributoNomeCompleto': [], 'Precision': [], 'Recall': [], 'F1': []}

    # Elenco dei concetti unici presenti nel GS
    all_concepts = gs_df['AttributoNomeCompleto'].unique()

    for concept in all_concepts:
        # Filtra i concetti corrispondenti nel GS e nelle Annotazioni
        gs_concepts = set(gs_df[gs_df['AttributoNomeCompleto'] == concept]['AttributoNomeCompleto'])
        ann_concepts = set(ann_df[ann_df['AttributoNomeCompleto'] == concept]['AttributoNomeCompleto'])

        # Calcolo della Precision
        precision = len(gs_concepts.intersection(ann_concepts)) / len(ann_concepts) if len(ann_concepts) > 0 else 0

        # Calcolo del Recall
        recall = len(gs_concepts.intersection(ann_concepts)) / len(gs_concepts) if len(gs_concepts) > 0 else 0

        # Calcolo del F1-Score
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Memorizza i risultati per ogni concetto
        metrics['AttributoNomeCompleto'].append(concept)
        metrics['Precision'].append(precision)
        metrics['Recall'].append(recall)
        metrics['F1'].append(f1)

    # Convertiamo in DataFrame per una migliore visualizzazione
    metrics_df = pd.DataFrame(metrics)

    return metrics_df

def calculate_aggregated_metrics(metrics_df):
    # Macro-average: media delle metriche
    precision_macro = metrics_df['Precision'].mean()
    recall_macro = metrics_df['Recall'].mean()
    f1_macro = metrics_df['F1'].mean()

    # Micro-average: somma dei veri positivi, falsi positivi e falsi negativi
    total_tp = sum([metrics_df.iloc[i]['Precision'] * metrics_df.iloc[i]['Recall'] for i in range(len(metrics_df))])
    total_fp = sum([metrics_df.iloc[i]['Precision'] * (1 - metrics_df.iloc[i]['Recall']) for i in range(len(metrics_df))])
    total_fn = sum([metrics_df.iloc[i]['Recall'] * (1 - metrics_df.iloc[i]['Precision']) for i in range(len(metrics_df))])

    precision_micro = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall_micro = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_micro = (2 * precision_micro * recall_micro) / (precision_micro + recall_micro) if (precision_micro + recall_micro) > 0 else 0

    # Restituisce le metriche aggregate
    aggregated_metrics = {
        'Precisionmacro': precision_macro,
        'Recallmacro': recall_macro,
        'F1macro': f1_macro,
        'Precisionmicro': precision_micro,
        'Recallmicro': recall_micro,
        'F1micro': f1_micro
    }

    return aggregated_metrics

# Calcoliamo le metriche per ciascun concetto (AttributoNomeCompleto)
metrics_df = calculate_metrics(gs_df, annotazioni)

# Calcoliamo le metriche aggregate (Macro e Micro)
aggregated_metrics = calculate_aggregated_metrics(metrics_df)

# Visualizziamo le metriche aggregate
print("Metriche Aggregate:")
for metric, value in aggregated_metrics.items():
    print(f"{metric}: {value}")


Metriche Aggregate:
Precisionmacro: 0.5540540540540541
Recallmacro: 0.5540540540540541
F1macro: 0.5540540540540541
Precisionmicro: 1.0
Recallmicro: 1.0
F1micro: 1.0


## Analisi della Similarit`a tra Attributi con Annotazioni Diverse

In [ ]:
import pandas as pd

# Funzione per calcolare l'intersezione tra due insiemi
def intersection(set1, set2):
    return len(set1.intersection(set2))

# Funzione per calcolare la similarità di Jaccard
def jaccard_similarity(set1, set2):
    return len(set1.intersection(set2)) / len(set1.union(set2)) if len(set1.union(set2)) > 0 else 0

# Funzione per calcolare il coefficiente di Sorensen-Dice
def sorensen_dice(set1, set2):
    return (2 * len(set1.intersection(set2))) / (len(set1) + len(set2)) if (len(set1) + len(set2)) > 0 else 0

# Funzione per calcolare la corrispondenza per ogni attributo
def calculate_similarity(gs_df, ann_df, threshold):
    similarity_results = {
        'AttributoNomeCompleto': [],
        'Jaccard': [],
        'Sorensen-Dice': [],
        'Corrispondenza': []
    }

    # Iteriamo su ciascun attributo
    for attribute in gs_df['AttributoNomeCompleto'].unique():
        # Prendiamo gli insiemi di annotazioni per l'attributo in entrambi gli schemi
        gs_annotations = set(gs_df[gs_df['AttributoNomeCompleto'] == attribute]['PrettyName'])
        ann_annotations = set(ann_df[ann_df['AttributoNomeCompleto'] == attribute]['PrettyName'])

        # Calcoliamo le metriche di similarità
        jaccard = jaccard_similarity(gs_annotations, ann_annotations)
        dice = sorensen_dice(gs_annotations, ann_annotations)

        # Verifica della corrispondenza (Jaccard)
        corrispondenza = 1 if jaccard >= threshold else 0

        # Aggiungiamo i risultati
        similarity_results['AttributoNomeCompleto'].append(attribute)
        similarity_results['Jaccard'].append(jaccard)
        similarity_results['Sorensen-Dice'].append(dice)
        similarity_results['Corrispondenza'].append(corrispondenza)

    # Creiamo un DataFrame con i risultati
    similarity_df = pd.DataFrame(similarity_results)

    return similarity_df



# Definiamo una soglia di similarità per la corrispondenza
threshold = 0.8

# Calcoliamo la similarità tra gli attributi
similarity_df = calculate_similarity(GS, annotazioni, threshold)

# Visualizziamo i risultati
print(similarity_df)


                 AttributoNomeCompleto   Jaccard  Sorensen-Dice  \
0                    location-location  0.000000       0.000000   
1                 location-location_id  0.043478       0.083333   
2                   location-address_1  0.040000       0.076923   
3                   location-address_2  0.038462       0.074074   
4                        location-city  0.080000       0.148148   
..                                 ...       ...            ...   
69     person-gender_source_concept_id  0.071429       0.133333   
70            person-race_source_value  0.081081       0.150000   
71       person-race_source_concept_id  0.034483       0.066667   
72       person-ethnicity_source_value  0.027027       0.052632   
73  person-ethnicity_source_concept_id  0.033333       0.064516   

    Corrispondenza  
0                0  
1                0  
2                0  
3                0  
4                0  
..             ...  
69               0  
70               0  
71    

## Valutazione della Corrispondenza con Precision, Recall e F1

In [ ]:
# Funzioni per calcolare TP, FP e FN
def true_positives(gs_set, ann_set):
    return len(gs_set.intersection(ann_set))

def false_positives(gs_set, ann_set):
    return len(ann_set - gs_set)

def false_negatives(gs_set, ann_set):
    return len(gs_set - ann_set)

# Funzione per calcolare Precision, Recall e F1-Score
def calculate_metrics(gs_set, ann_set):
    tp = true_positives(gs_set, ann_set)
    fp = false_positives(gs_set, ann_set)
    fn = false_negatives(gs_set, ann_set)

    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return precision, recall, f1

# Funzione per confrontare gli attributi e calcolare le metriche aggregate
def compare_attributes(gs_df, ann_df, threshold=0.5):
    comparison_results = {}

    for attribute in gs_df['AttributoNomeCompleto'].unique():
        # Estrai gli insiemi di annotazioni per l'attributo corrente
        gs_annotations = set(gs_df[gs_df['AttributoNomeCompleto'] == attribute]['PrettyName'])
        ann_annotations = set(ann_df[ann_df['AttributoNomeCompleto'] == attribute]['PrettyName'])

        # Calcolo delle metriche di Precision, Recall e F1-Score
        precision, recall, f1 = calculate_metrics(gs_annotations, ann_annotations)

        # Salva i risultati
        comparison_results[attribute] = {
            'Precision': precision,
            'Recall': recall,
            'F1': f1
        }

    return comparison_results

# Supponiamo che gs_df e ann_df siano i tuoi DataFrame contenenti i dati per GS e Annotator X
# gs_df = pd.read_csv('gold_standard.csv')
# ann_df = pd.read_csv('annotator.csv')

# Confrontiamo gli attributi
comparison_results = compare_attributes(GS, annotazioni)

# Visualizziamo i risultati
for attribute, metrics in comparison_results.items():
    print(f"Attributo: {attribute}")
    print(f"Precision: {metrics['Precision']}")
    print(f"Recall: {metrics['Recall']}")
    print(f"F1: {metrics['F1']}")
    print()

Attributo: location-location
Precision: 0
Recall: 0.0
F1: 0

Attributo: location-location_id
Precision: 0.045454545454545456
Recall: 0.5
F1: 0.08333333333333334

Attributo: location-address_1
Precision: 0.04
Recall: 1.0
F1: 0.07692307692307693

Attributo: location-address_2
Precision: 0.038461538461538464
Recall: 1.0
F1: 0.07407407407407407

Attributo: location-city
Precision: 0.08
Recall: 1.0
F1: 0.14814814814814814

Attributo: location-state
Precision: 0.041666666666666664
Recall: 0.5
F1: 0.07692307692307693

Attributo: location-zip
Precision: 0.09090909090909091
Recall: 0.6666666666666666
F1: 0.16

Attributo: location-county
Precision: 0.10526315789473684
Recall: 1.0
F1: 0.1904761904761905

Attributo: location-country
Precision: 0
Recall: 0.0
F1: 0

Attributo: location-location_source_value
Precision: 0.08695652173913043
Recall: 1.0
F1: 0.16

Attributo: location-latitude
Precision: 0
Recall: 0.0
F1: 0

Attributo: location-longitude
Precision: 0
Recall: 0.0
F1: 0

Attributo: cost-cos